# Preprocessing static variables

In [82]:
import pandas as pd
import datetime
pd.options.mode.chained_assignment = None  # default='warn'
import math

In [2]:
mimic_static = pd.read_csv("./data/mimic_dems.csv")
mimic_static.head()

,hadm_id,admittime,dischtime,deathtime,admission_type,admission_location,discharge_location,insurance,language,religion,marital_status,ethnicity,hospital_expire_flag,icd9_code,short_title,subject_id,gender,dob,dod,los
0,174105,2194-06-13 18:39:00,2194-06-18 16:50:00,NaN,EMERGENCY,EMERGENCY ROOM ADMIT,HOME HEALTH CARE,Medicare,NaN,JEWISH,MARRIED,WHITE,0,53100,Ac stomach ulcer w hem,112,M,1894-06-13 00:00:00,2196-09-29 00:00:00,0.8402
1,174105,2194-06-13 18:39:00,2194-06-18 16:50:00,NaN,EMERGENCY,EMERGENCY ROOM ADMIT,HOME HEALTH CARE,Medicare,NaN,JEWISH,MARRIED,WHITE,0,41071,"Subendo infarct, initial",112,M,1894-06-13 00:00:00,2196-09-29 00:00:00,0.8402
2,174105,2194-06-13 18:39:00,2194-06-18 16:50:00,NaN,EMERGENCY,EMERGENCY ROOM ADMIT,HOME HEALTH CARE,Medicare,NaN,JEWISH,MARRIED,WHITE,0,2859,Anemia NOS,112,M,1894-06-13 00:00:00,2196-09-29 00:00:00,0.8402
3,174105,2194-06-13 18:39:00,2194-06-18 16:50:00,NaN,EMERGENCY,EMERGENCY ROOM ADMIT,HOME HEALTH CARE,Medicare,NaN,JEWISH,MARRIED,WHITE,0,41401,Crnry athrscl natve vssl,112,M,1894-06-13 00:00:00,2196-09-29 00:00:00,0.8402
4,174105,2194-06-13 18:39:00,2194-06-18 16:50:00,NaN,EMERGENCY,EMERGENCY ROOM ADMIT,HOME HEALTH CARE,Medicare,NaN,JEWISH,MARRIED,WHITE,0,725,Polymyalgia rheumatica,112,M,1894-06-13 00:00:00,2196-09-29 00:00:00,0.8402


In [3]:
lab_cv_hadm_id = set(pd.read_csv("../Processed_Data/mimic_lab_CV_reg_pre.csv")["hadm_id"])
chart_cv_only_num_hadm_id = set(pd.read_csv("../Processed_Data/mimic_chart_CV_only_numeric.csv")["hadm_id"])
chart_cv_only_cat_hadm_id = set(pd.read_csv("../Processed_Data/mimic_chart_CV_only_cat.csv")["hadm_id"])
mimic_static_CV_hadm_id = lab_cv_hadm_id.intersection(chart_cv_only_num_hadm_id)
mimic_static_CV_hadm_id = mimic_static_CV_hadm_id.intersection(chart_cv_only_cat_hadm_id)
print(len(mimic_static_CV_hadm_id))


/Users/shutingcui/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3165: DtypeWarning: Columns (10,13) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


26066


In [5]:
lab_mv_hadm_id = set(pd.read_csv("../Processed_Data/mimic_lab_MV_reg_pre.csv")["hadm_id"])
chart_mv_only_num_hadm_id = set(pd.read_csv("../Processed_Data/mimic_chart_MV_only_numeric.csv")["hadm_id"])
chart_mv_only_cat_hadm_id = set(pd.read_csv("../Processed_Data/mimic_chart_MV_only_cat.csv")["hadm_id"])
mimic_static_MV_hadm_id = lab_mv_hadm_id.intersection(chart_mv_only_num_hadm_id)
mimic_static_MV_hadm_id = mimic_static_MV_hadm_id.intersection(chart_mv_only_cat_hadm_id)
print(len(mimic_static_MV_hadm_id))


21018


## a) divide mimic_static into 2 parts: mimic_static_CV and mimic_static_MV based on above "mimic_static_CV_hadm_id" and "mimic_static_MV_hadm_id"

In [6]:
mimic_static_CV = mimic_static[mimic_static["hadm_id"].isin(mimic_static_CV_hadm_id)]
mimic_static_MV = mimic_static[mimic_static["hadm_id"].isin(mimic_static_MV_hadm_id)]
mimic_static_CV.reset_index(drop=True, inplace=True)
mimic_static_MV.reset_index(drop=True, inplace=True)
print(mimic_static.shape)
print(mimic_static_CV.shape)
print(mimic_static_MV.shape)

(714344, 20)
(285351, 20)
(341065, 20)


## b) transfer the icd_9 codes into 1-19 categorical variable

In [8]:
def ICD9_codes_trans(org_icd9):
    if pd.isna(org_icd9):
        return np.nan
    else:
        first_3_num = org_icd9[0:3]
        transformed_icd9 = 0
        if first_3_num[0] == 'V':
            transformed_icd9 = 19
        elif first_3_num[0] == 'E':
            transformed_icd9 = 20
        else:
            code_range = float(first_3_num)
            if code_range >= 1 and code_range <= 139:
                transformed_icd9 = 1
            if code_range >= 140 and code_range <= 239:
                transformed_icd9 = 2
            if code_range >= 240 and code_range <= 279:
                transformed_icd9 = 3
            if code_range >= 280 and code_range <= 289:
                transformed_icd9 = 4
            if code_range >= 290 and code_range <= 319:
                transformed_icd9 = 5
            if code_range >= 320 and code_range <= 389:
                transformed_icd9 = 6
            if code_range >= 390 and code_range <= 459:
                transformed_icd9 = 7
            if code_range >= 460 and code_range <= 519:
                transformed_icd9 = 8
            if code_range >= 520 and code_range <= 579:
                transformed_icd9 = 9
            if code_range >= 580 and code_range <= 629:
                transformed_icd9 = 10
            if code_range >= 630 and code_range <= 679:
                transformed_icd9 = 11
            if code_range >= 680 and code_range <= 709:
                transformed_icd9 = 12
            if code_range >= 710 and code_range <= 739:
                transformed_icd9 = 13
            if code_range >= 740 and code_range <= 759:
                transformed_icd9 = 14
            if code_range >= 780 and code_range <= 789:
                transformed_icd9 = 15
            if code_range >= 790 and code_range <= 796:
                transformed_icd9 = 16
            if code_range >= 797 and code_range <= 799:
                transformed_icd9 = 17
            if code_range >= 800 and code_range <= 999:
                transformed_icd9 = 18
            
    return int(transformed_icd9)  

* transfer on "mimic_static_CV "

In [9]:
mimic_static_CV.head()

,hadm_id,admittime,dischtime,deathtime,admission_type,admission_location,discharge_location,insurance,language,religion,marital_status,ethnicity,hospital_expire_flag,icd9_code,short_title,subject_id,gender,dob,dod,los
0,141647,2183-08-20 17:25:00,2183-08-24 15:28:00,NaN,EMERGENCY,CLINIC REFERRAL/PREMATURE,HOME,Private,NaN,CATHOLIC,SINGLE,WHITE,0,19889,Secondary malig neo NEC,127,F,2135-02-01 00:00:00,2184-02-22 00:00:00,3.9174
1,141647,2183-08-20 17:25:00,2183-08-24 15:28:00,NaN,EMERGENCY,CLINIC REFERRAL/PREMATURE,HOME,Private,NaN,CATHOLIC,SINGLE,WHITE,0,1972,Second malig neo pleura,127,F,2135-02-01 00:00:00,2184-02-22 00:00:00,3.9174
2,141647,2183-08-20 17:25:00,2183-08-24 15:28:00,NaN,EMERGENCY,CLINIC REFERRAL/PREMATURE,HOME,Private,NaN,CATHOLIC,SINGLE,WHITE,0,V103,Hx of breast malignancy,127,F,2135-02-01 00:00:00,2184-02-22 00:00:00,3.9174
3,172056,2143-03-03 09:25:00,2143-03-18 13:01:00,NaN,EMERGENCY,EMERGENCY ROOM ADMIT,REHAB/DISTINCT PART HOSP,Medicaid,ENGL,OTHER,SINGLE,BLACK/AFRICAN AMERICAN,0,03811,Meth susc Staph aur sept,64,F,2116-06-27 00:00:00,NaN,2.3250
4,172056,2143-03-03 09:25:00,2143-03-18 13:01:00,NaN,EMERGENCY,EMERGENCY ROOM ADMIT,REHAB/DISTINCT PART HOSP,Medicaid,ENGL,OTHER,SINGLE,BLACK/AFRICAN AMERICAN,0,99591,Sepsis,64,F,2116-06-27 00:00:00,NaN,2.3250


In [10]:
mimic_static_CV[["transformed_icd9"]] = mimic_static_CV[["icd9_code"]].apply((lambda x: ICD9_codes_trans(x["icd9_code"])), axis=1)
mimic_static_CV[["transformed_icd9"]] = mimic_static_CV[["transformed_icd9"]].fillna(-1)
mimic_static_CV[["transformed_icd9"]] = mimic_static_CV[["transformed_icd9"]].astype(int)

with pd.option_context('display.max_rows', None, 'display.max_columns', None): 
    display(mimic_static_CV.head())
    

,hadm_id,admittime,dischtime,deathtime,admission_type,admission_location,discharge_location,insurance,language,religion,marital_status,ethnicity,hospital_expire_flag,icd9_code,short_title,subject_id,gender,dob,dod,los,transformed_icd9
0,141647,2183-08-20 17:25:00,2183-08-24 15:28:00,NaN,EMERGENCY,CLINIC REFERRAL/PREMATURE,HOME,Private,NaN,CATHOLIC,SINGLE,WHITE,0,19889,Secondary malig neo NEC,127,F,2135-02-01 00:00:00,2184-02-22 00:00:00,3.9174,2
1,141647,2183-08-20 17:25:00,2183-08-24 15:28:00,NaN,EMERGENCY,CLINIC REFERRAL/PREMATURE,HOME,Private,NaN,CATHOLIC,SINGLE,WHITE,0,1972,Second malig neo pleura,127,F,2135-02-01 00:00:00,2184-02-22 00:00:00,3.9174,2
2,141647,2183-08-20 17:25:00,2183-08-24 15:28:00,NaN,EMERGENCY,CLINIC REFERRAL/PREMATURE,HOME,Private,NaN,CATHOLIC,SINGLE,WHITE,0,V103,Hx of breast malignancy,127,F,2135-02-01 00:00:00,2184-02-22 00:00:00,3.9174,19
3,172056,2143-03-03 09:25:00,2143-03-18 13:01:00,NaN,EMERGENCY,EMERGENCY ROOM ADMIT,REHAB/DISTINCT PART HOSP,Medicaid,ENGL,OTHER,SINGLE,BLACK/AFRICAN AMERICAN,0,03811,Meth susc Staph aur sept,64,F,2116-06-27 00:00:00,NaN,2.3250,1
4,172056,2143-03-03 09:25:00,2143-03-18 13:01:00,NaN,EMERGENCY,EMERGENCY ROOM ADMIT,REHAB/DISTINCT PART HOSP,Medicaid,ENGL,OTHER,SINGLE,BLACK/AFRICAN AMERICAN,0,99591,Sepsis,64,F,2116-06-27 00:00:00,NaN,2.3250,18


* transfer on "mimic_static_MV "

In [11]:
mimic_static_MV[["transformed_icd9"]] = mimic_static_MV[["icd9_code"]].apply((lambda x: ICD9_codes_trans(x["icd9_code"])), axis=1)
mimic_static_MV[["transformed_icd9"]] = mimic_static_MV[["transformed_icd9"]].fillna(-1)
mimic_static_MV[["transformed_icd9"]] = mimic_static_MV[["transformed_icd9"]].astype(int)

with pd.option_context('display.max_rows', None, 'display.max_columns', None): 
    display(mimic_static_MV.head())
    

,hadm_id,admittime,dischtime,deathtime,admission_type,admission_location,discharge_location,insurance,language,religion,marital_status,ethnicity,hospital_expire_flag,icd9_code,short_title,subject_id,gender,dob,dod,los,transformed_icd9
0,165660,2134-05-10 11:30:00,2134-05-20 13:16:00,NaN,ELECTIVE,PHYS REFERRAL/NORMAL DELI,LONG TERM CARE HOSPITAL,Medicare,ENGL,NOT SPECIFIED,MARRIED,WHITE,0,55321,Incisional hernia,36,M,2061-08-17 00:00:00,NaN,4.3483,9
1,165660,2134-05-10 11:30:00,2134-05-20 13:16:00,NaN,ELECTIVE,PHYS REFERRAL/NORMAL DELI,LONG TERM CARE HOSPITAL,Medicare,ENGL,NOT SPECIFIED,MARRIED,WHITE,0,41511,Iatrogen pulm emb/infarc,36,M,2061-08-17 00:00:00,NaN,4.3483,7
2,165660,2134-05-10 11:30:00,2134-05-20 13:16:00,NaN,ELECTIVE,PHYS REFERRAL/NORMAL DELI,LONG TERM CARE HOSPITAL,Medicare,ENGL,NOT SPECIFIED,MARRIED,WHITE,0,5185,NaN,36,M,2061-08-17 00:00:00,NaN,4.3483,8
3,165660,2134-05-10 11:30:00,2134-05-20 13:16:00,NaN,ELECTIVE,PHYS REFERRAL/NORMAL DELI,LONG TERM CARE HOSPITAL,Medicare,ENGL,NOT SPECIFIED,MARRIED,WHITE,0,486,"Pneumonia, organism NOS",36,M,2061-08-17 00:00:00,NaN,4.3483,8
4,165660,2134-05-10 11:30:00,2134-05-20 13:16:00,NaN,ELECTIVE,PHYS REFERRAL/NORMAL DELI,LONG TERM CARE HOSPITAL,Medicare,ENGL,NOT SPECIFIED,MARRIED,WHITE,0,99739,Respiratory comp NEC,36,M,2061-08-17 00:00:00,NaN,4.3483,18


## e) divide column "transformed_icd9" into 20 catogoricals: icd9_1 - icd9_20**


* transfer on mimic_static_CV

In [12]:
icd9_categories_CV = pd.get_dummies(mimic_static_CV.transformed_icd9)


In [13]:
new_columns = []
for i in range(1,21):
    new_columns.append("icd9_" + str(i))
icd9_categories_CV.columns = new_columns
mimic_static_CV = pd.concat([mimic_static_CV, icd9_categories_CV], axis=1)
mimic_static_CV.drop(['transformed_icd9', "icd9_code"], axis=1, inplace=True)
with pd.option_context('display.max_rows', None, 'display.max_columns', None): 
    display(mimic_static_CV.head())


,hadm_id,admittime,dischtime,deathtime,admission_type,admission_location,discharge_location,insurance,language,religion,marital_status,ethnicity,hospital_expire_flag,short_title,subject_id,gender,dob,dod,los,icd9_1,icd9_2,icd9_3,icd9_4,icd9_5,icd9_6,icd9_7,icd9_8,icd9_9,icd9_10,icd9_11,icd9_12,icd9_13,icd9_14,icd9_15,icd9_16,icd9_17,icd9_18,icd9_19,icd9_20
0,141647,2183-08-20 17:25:00,2183-08-24 15:28:00,NaN,EMERGENCY,CLINIC REFERRAL/PREMATURE,HOME,Private,NaN,CATHOLIC,SINGLE,WHITE,0,Secondary malig neo NEC,127,F,2135-02-01 00:00:00,2184-02-22 00:00:00,3.9174,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,141647,2183-08-20 17:25:00,2183-08-24 15:28:00,NaN,EMERGENCY,CLINIC REFERRAL/PREMATURE,HOME,Private,NaN,CATHOLIC,SINGLE,WHITE,0,Second malig neo pleura,127,F,2135-02-01 00:00:00,2184-02-22 00:00:00,3.9174,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,141647,2183-08-20 17:25:00,2183-08-24 15:28:00,NaN,EMERGENCY,CLINIC REFERRAL/PREMATURE,HOME,Private,NaN,CATHOLIC,SINGLE,WHITE,0,Hx of breast malignancy,127,F,2135-02-01 00:00:00,2184-02-22 00:00:00,3.9174,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
3,172056,2143-03-03 09:25:00,2143-03-18 13:01:00,NaN,EMERGENCY,EMERGENCY ROOM ADMIT,REHAB/DISTINCT PART HOSP,Medicaid,ENGL,OTHER,SINGLE,BLACK/AFRICAN AMERICAN,0,Meth susc Staph aur sept,64,F,2116-06-27 00:00:00,NaN,2.3250,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,172056,2143-03-03 09:25:00,2143-03-18 13:01:00,NaN,EMERGENCY,EMERGENCY ROOM ADMIT,REHAB/DISTINCT PART HOSP,Medicaid,ENGL,OTHER,SINGLE,BLACK/AFRICAN AMERICAN,0,Sepsis,64,F,2116-06-27 00:00:00,NaN,2.3250,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0


* transfer on mimic_static_MV

## For mimic_static_MV, there is a row with "icd9_code" == 76076, the first three number "760" is not included in the cases function "ICD9_codes_trans" tackles. Just delete this row

In [14]:
mimic_static_MV[mimic_static_MV["icd9_code"] == "76076"]

,hadm_id,admittime,dischtime,deathtime,admission_type,admission_location,discharge_location,insurance,language,religion,...,ethnicity,hospital_expire_flag,icd9_code,short_title,subject_id,gender,dob,dod,los,transformed_icd9
133654,140225,2163-11-05 02:37:00,2163-11-17 13:51:00,NaN,EMERGENCY,CLINIC REFERRAL/PREMATURE,HOME HEALTH CARE,Private,ENGL,JEWISH,...,WHITE,0,76076,Maternal DES af NB/fetus,66780,F,2108-02-05 00:00:00,NaN,2.9018,0


In [17]:
print(mimic_static_MV.shape)
mimic_static_MV.drop(mimic_static_MV[mimic_static_MV["icd9_code"] == "76076"].index, axis=0, inplace=True)
print(mimic_static_MV.shape)

(341065, 21)
(341064, 21)


In [18]:
mimic_static_MV.reset_index(drop=True, inplace=True)

In [19]:
# To check whether successfully delete the row with "icd9_code" == 76076
mimic_static_MV[mimic_static_MV["icd9_code"] == "76076"]

,hadm_id,admittime,dischtime,deathtime,admission_type,admission_location,discharge_location,insurance,language,religion,...,ethnicity,hospital_expire_flag,icd9_code,short_title,subject_id,gender,dob,dod,los,transformed_icd9


In [20]:
mimic_static_MV.head()

,hadm_id,admittime,dischtime,deathtime,admission_type,admission_location,discharge_location,insurance,language,religion,...,ethnicity,hospital_expire_flag,icd9_code,short_title,subject_id,gender,dob,dod,los,transformed_icd9
0,165660,2134-05-10 11:30:00,2134-05-20 13:16:00,NaN,ELECTIVE,PHYS REFERRAL/NORMAL DELI,LONG TERM CARE HOSPITAL,Medicare,ENGL,NOT SPECIFIED,...,WHITE,0,55321,Incisional hernia,36,M,2061-08-17 00:00:00,NaN,4.3483,9
1,165660,2134-05-10 11:30:00,2134-05-20 13:16:00,NaN,ELECTIVE,PHYS REFERRAL/NORMAL DELI,LONG TERM CARE HOSPITAL,Medicare,ENGL,NOT SPECIFIED,...,WHITE,0,41511,Iatrogen pulm emb/infarc,36,M,2061-08-17 00:00:00,NaN,4.3483,7
2,165660,2134-05-10 11:30:00,2134-05-20 13:16:00,NaN,ELECTIVE,PHYS REFERRAL/NORMAL DELI,LONG TERM CARE HOSPITAL,Medicare,ENGL,NOT SPECIFIED,...,WHITE,0,5185,NaN,36,M,2061-08-17 00:00:00,NaN,4.3483,8
3,165660,2134-05-10 11:30:00,2134-05-20 13:16:00,NaN,ELECTIVE,PHYS REFERRAL/NORMAL DELI,LONG TERM CARE HOSPITAL,Medicare,ENGL,NOT SPECIFIED,...,WHITE,0,486,"Pneumonia, organism NOS",36,M,2061-08-17 00:00:00,NaN,4.3483,8
4,165660,2134-05-10 11:30:00,2134-05-20 13:16:00,NaN,ELECTIVE,PHYS REFERRAL/NORMAL DELI,LONG TERM CARE HOSPITAL,Medicare,ENGL,NOT SPECIFIED,...,WHITE,0,99739,Respiratory comp NEC,36,M,2061-08-17 00:00:00,NaN,4.3483,18


In [21]:
icd9_categories_MV = pd.get_dummies(mimic_static_MV.transformed_icd9)

In [22]:
icd9_categories_MV.columns = new_columns
mimic_static_MV = pd.concat([mimic_static_MV, icd9_categories_MV], axis=1)
mimic_static_MV.drop(['transformed_icd9', "icd9_code"], axis=1, inplace=True)
with pd.option_context('display.max_rows', None, 'display.max_columns', None): 
    display(mimic_static_MV.head())

,hadm_id,admittime,dischtime,deathtime,admission_type,admission_location,discharge_location,insurance,language,religion,marital_status,ethnicity,hospital_expire_flag,short_title,subject_id,gender,dob,dod,los,icd9_1,icd9_2,icd9_3,icd9_4,icd9_5,icd9_6,icd9_7,icd9_8,icd9_9,icd9_10,icd9_11,icd9_12,icd9_13,icd9_14,icd9_15,icd9_16,icd9_17,icd9_18,icd9_19,icd9_20
0,165660,2134-05-10 11:30:00,2134-05-20 13:16:00,NaN,ELECTIVE,PHYS REFERRAL/NORMAL DELI,LONG TERM CARE HOSPITAL,Medicare,ENGL,NOT SPECIFIED,MARRIED,WHITE,0,Incisional hernia,36,M,2061-08-17 00:00:00,NaN,4.3483,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
1,165660,2134-05-10 11:30:00,2134-05-20 13:16:00,NaN,ELECTIVE,PHYS REFERRAL/NORMAL DELI,LONG TERM CARE HOSPITAL,Medicare,ENGL,NOT SPECIFIED,MARRIED,WHITE,0,Iatrogen pulm emb/infarc,36,M,2061-08-17 00:00:00,NaN,4.3483,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
2,165660,2134-05-10 11:30:00,2134-05-20 13:16:00,NaN,ELECTIVE,PHYS REFERRAL/NORMAL DELI,LONG TERM CARE HOSPITAL,Medicare,ENGL,NOT SPECIFIED,MARRIED,WHITE,0,NaN,36,M,2061-08-17 00:00:00,NaN,4.3483,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
3,165660,2134-05-10 11:30:00,2134-05-20 13:16:00,NaN,ELECTIVE,PHYS REFERRAL/NORMAL DELI,LONG TERM CARE HOSPITAL,Medicare,ENGL,NOT SPECIFIED,MARRIED,WHITE,0,"Pneumonia, organism NOS",36,M,2061-08-17 00:00:00,NaN,4.3483,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
4,165660,2134-05-10 11:30:00,2134-05-20 13:16:00,NaN,ELECTIVE,PHYS REFERRAL/NORMAL DELI,LONG TERM CARE HOSPITAL,Medicare,ENGL,NOT SPECIFIED,MARRIED,WHITE,0,Respiratory comp NEC,36,M,2061-08-17 00:00:00,NaN,4.3483,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0


## g) combine rows with same hadm_id into one row by moving all the 1 values in columns "icd9_xx" into the first row of the same-adm_id-group


* transfer on mimic_static_CV

## f) **drop column "hospital_expire_flag" & "short title"because they're redundant with "discharge_location"**
## drop columns "subject_id", "los" which is usless for generating the static variables and will cause redundant rows for the same hadm_id (because of the difference in columns "subject_id" and "los"(length of stay))

In [24]:
all_hadm_id_unique_CV = list(mimic_static_CV["hadm_id"].unique())
new_mimic_CV_gb_adm_id = mimic_static_CV.groupby("hadm_id")
final_formatted_static_CV = 0
for i, item in enumerate(all_hadm_id_unique_CV):
#     if (i+1) % 10000 == 0:
#         print("current transform patient: ", i+1)
#         print("current final_formatted_static_var's shape: ", final_formatted_static_var.shape)
    cur_patient = new_mimic_CV_gb_adm_id.get_group(item)
    cur_patient_icd9_cols = cur_patient[new_columns]
    for j in range(20):
        if (cur_patient_icd9_cols.iloc[:, j]==0).all():
            continue
        else:
            cur_patient_icd9_cols.iloc[0, j] = 1
# dropping cols "hospital_expire_flag", "short_title", "subject_id", and "los" is for successfully dropping duplicates later
    cur_patient.drop(columns = new_columns+["hospital_expire_flag", "short_title", "subject_id", "los"], axis=1, inplace=True)
    cur_patient.drop_duplicates(inplace=True)
    cur_combined_icd9_codes = cur_patient_icd9_cols.iloc[[0]]
    cur_patient_formatted_static_CV = pd.concat([cur_patient, cur_combined_icd9_codes], axis=1)
    if i == 0:
        final_formatted_static_CV = cur_patient_formatted_static_CV
    else:
        final_formatted_static_CV = pd.concat([final_formatted_static_CV, cur_patient_formatted_static_CV], axis=0)


In [25]:
final_formatted_static_CV.reset_index(drop=True, inplace=True)

In [26]:
final_formatted_static_CV.shape

(26066, 35)

In [27]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None): 
    display(final_formatted_static_CV.head())

,hadm_id,admittime,dischtime,deathtime,admission_type,admission_location,discharge_location,insurance,language,religion,marital_status,ethnicity,gender,dob,dod,icd9_1,icd9_2,icd9_3,icd9_4,icd9_5,icd9_6,icd9_7,icd9_8,icd9_9,icd9_10,icd9_11,icd9_12,icd9_13,icd9_14,icd9_15,icd9_16,icd9_17,icd9_18,icd9_19,icd9_20
0,141647,2183-08-20 17:25:00,2183-08-24 15:28:00,NaN,EMERGENCY,CLINIC REFERRAL/PREMATURE,HOME,Private,NaN,CATHOLIC,SINGLE,WHITE,F,2135-02-01 00:00:00,2184-02-22 00:00:00,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
1,172056,2143-03-03 09:25:00,2143-03-18 13:01:00,NaN,EMERGENCY,EMERGENCY ROOM ADMIT,REHAB/DISTINCT PART HOSP,Medicaid,ENGL,OTHER,SINGLE,BLACK/AFRICAN AMERICAN,F,2116-06-27 00:00:00,NaN,1,0,0,0,1,0,1,1,0,0,0,0,1,0,1,0,0,1,0,0
2,142768,2164-11-18 07:00:00,2164-11-20 14:30:00,NaN,EMERGENCY,TRANSFER FROM HOSP/EXTRAM,HOME,Private,NaN,CATHOLIC,MARRIED,UNKNOWN/NOT SPECIFIED,M,2119-05-12 00:00:00,NaN,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
3,158569,2142-04-01 12:34:00,2142-04-08 14:46:00,NaN,URGENT,TRANSFER FROM HOSP/EXTRAM,HOME HEALTH CARE,Medicare,NaN,UNOBTAINABLE,MARRIED,UNKNOWN/NOT SPECIFIED,F,2076-09-15 00:00:00,2142-11-20 00:00:00,0,0,1,0,0,0,1,0,1,0,0,1,0,0,0,0,0,0,0,0
4,170324,2156-03-31 16:11:00,2156-04-29 15:05:00,NaN,EMERGENCY,EMERGENCY ROOM ADMIT,HOME,Government,ENGL,CATHOLIC,MARRIED,WHITE,M,2117-10-02 00:00:00,NaN,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,1,1,0


* transfer on mimic_static_MV

In [28]:
all_hadm_id_unique_MV = list(mimic_static_MV["hadm_id"].unique())
new_mimic_MV_gb_adm_id = mimic_static_MV.groupby("hadm_id")
final_formatted_static_MV = 0
for i, item in enumerate(all_hadm_id_unique_MV):
#     if (i+1) % 10000 == 0:
#         print("current transform patient: ", i+1)
#         print("current final_formatted_static_var's shape: ", final_formatted_static_var.shape)
    cur_patient = new_mimic_MV_gb_adm_id.get_group(item)
    cur_patient_icd9_cols = cur_patient[new_columns]
    for j in range(20):
        if (cur_patient_icd9_cols.iloc[:, j]==0).all():
            continue
        else:
            cur_patient_icd9_cols.iloc[0, j] = 1
    cur_patient.drop(columns = new_columns+["hospital_expire_flag", "short_title", "subject_id", "los"], axis=1, inplace=True)
    cur_patient.drop_duplicates(inplace=True)
    cur_combined_icd9_codes = cur_patient_icd9_cols.iloc[[0]]
    cur_patient_formatted_static_MV = pd.concat([cur_patient, cur_combined_icd9_codes], axis=1)
    if i == 0:
        final_formatted_static_MV = cur_patient_formatted_static_MV
    else:
        final_formatted_static_MV = pd.concat([final_formatted_static_MV, cur_patient_formatted_static_MV], axis=0)


In [29]:
final_formatted_static_MV.reset_index(drop=True, inplace=True)

In [30]:
final_formatted_static_MV.shape

(21018, 35)

In [31]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None): 
    display(final_formatted_static_MV.head())

,hadm_id,admittime,dischtime,deathtime,admission_type,admission_location,discharge_location,insurance,language,religion,marital_status,ethnicity,gender,dob,dod,icd9_1,icd9_2,icd9_3,icd9_4,icd9_5,icd9_6,icd9_7,icd9_8,icd9_9,icd9_10,icd9_11,icd9_12,icd9_13,icd9_14,icd9_15,icd9_16,icd9_17,icd9_18,icd9_19,icd9_20
0,165660,2134-05-10 11:30:00,2134-05-20 13:16:00,NaN,ELECTIVE,PHYS REFERRAL/NORMAL DELI,LONG TERM CARE HOSPITAL,Medicare,ENGL,NOT SPECIFIED,MARRIED,WHITE,M,2061-08-17 00:00:00,NaN,0,0,0,0,1,0,1,1,1,1,0,0,0,0,0,0,0,1,1,0
1,126055,2141-10-13 23:10:00,2141-11-03 18:45:00,NaN,EMERGENCY,EMERGENCY ROOM ADMIT,HOME,Medicaid,ENGL,NOT SPECIFIED,SINGLE,BLACK/AFRICAN AMERICAN,F,2117-08-07 00:00:00,2142-08-30 00:00:00,0,0,1,1,0,1,1,1,1,1,0,0,1,0,1,1,0,0,1,0
2,136572,2142-06-18 03:11:00,2142-06-20 16:34:00,NaN,EMERGENCY,EMERGENCY ROOM ADMIT,HOME HEALTH CARE,Medicaid,ENGL,NOT SPECIFIED,SINGLE,BLACK/AFRICAN AMERICAN,F,2117-08-07 00:00:00,2142-08-30 00:00:00,0,0,1,1,1,1,1,0,1,1,0,0,1,0,1,0,0,0,1,0
3,137510,2142-04-20 13:39:00,2142-04-22 15:30:00,NaN,EMERGENCY,EMERGENCY ROOM ADMIT,HOME HEALTH CARE,Medicaid,ENGL,NOT SPECIFIED,SINGLE,BLACK/AFRICAN AMERICAN,F,2117-08-07 00:00:00,2142-08-30 00:00:00,0,0,1,1,0,1,1,0,0,1,0,0,1,0,1,0,0,0,1,0
4,158943,2142-03-21 09:25:00,2142-03-23 18:11:00,NaN,EMERGENCY,EMERGENCY ROOM ADMIT,HOME,Medicaid,ENGL,NOT SPECIFIED,SINGLE,BLACK/AFRICAN AMERICAN,F,2117-08-07 00:00:00,2142-08-30 00:00:00,1,0,1,1,0,1,1,0,0,1,0,0,1,0,1,1,0,1,1,1


In [32]:
len(final_formatted_static_CV["hadm_id"].unique())

26066

In [33]:
len(final_formatted_static_MV["hadm_id"].unique())

21018

## c) transfer the variable "dob" into the age computed from the day of birth to the admission time using "admittime" in mimic_static

* Transfer on static_CV

In [34]:
final_formatted_static_CV[["adm_year"]] = final_formatted_static_CV[["admittime"]].apply((lambda x: datetime.datetime.strptime(x["admittime"], "%Y-%m-%d %H:%M:%S").year), axis=1)
final_formatted_static_CV[["dob_year"]] = final_formatted_static_CV[["dob"]].apply((lambda x: datetime.datetime.strptime(x["dob"], "%Y-%m-%d %H:%M:%S").year), axis=1)
final_formatted_static_CV[["age"]] = final_formatted_static_CV["adm_year"] - final_formatted_static_CV["dob_year"]
final_formatted_static_CV.head()

,hadm_id,admittime,dischtime,deathtime,admission_type,admission_location,discharge_location,insurance,language,religion,...,icd9_14,icd9_15,icd9_16,icd9_17,icd9_18,icd9_19,icd9_20,adm_year,dob_year,age
0,141647,2183-08-20 17:25:00,2183-08-24 15:28:00,NaN,EMERGENCY,CLINIC REFERRAL/PREMATURE,HOME,Private,NaN,CATHOLIC,...,0,0,0,0,0,1,0,2183,2135,48
1,172056,2143-03-03 09:25:00,2143-03-18 13:01:00,NaN,EMERGENCY,EMERGENCY ROOM ADMIT,REHAB/DISTINCT PART HOSP,Medicaid,ENGL,OTHER,...,0,1,0,0,1,0,0,2143,2116,27
2,142768,2164-11-18 07:00:00,2164-11-20 14:30:00,NaN,EMERGENCY,TRANSFER FROM HOSP/EXTRAM,HOME,Private,NaN,CATHOLIC,...,0,0,0,0,0,0,0,2164,2119,45
3,158569,2142-04-01 12:34:00,2142-04-08 14:46:00,NaN,URGENT,TRANSFER FROM HOSP/EXTRAM,HOME HEALTH CARE,Medicare,NaN,UNOBTAINABLE,...,0,0,0,0,0,0,0,2142,2076,66
4,170324,2156-03-31 16:11:00,2156-04-29 15:05:00,NaN,EMERGENCY,EMERGENCY ROOM ADMIT,HOME,Government,ENGL,CATHOLIC,...,0,0,0,0,1,1,0,2156,2117,39


In [35]:
final_formatted_static_CV.drop(columns=["adm_year", "dob_year"], inplace=True)
final_formatted_static_CV.head()

,hadm_id,admittime,dischtime,deathtime,admission_type,admission_location,discharge_location,insurance,language,religion,...,icd9_12,icd9_13,icd9_14,icd9_15,icd9_16,icd9_17,icd9_18,icd9_19,icd9_20,age
0,141647,2183-08-20 17:25:00,2183-08-24 15:28:00,NaN,EMERGENCY,CLINIC REFERRAL/PREMATURE,HOME,Private,NaN,CATHOLIC,...,0,0,0,0,0,0,0,1,0,48
1,172056,2143-03-03 09:25:00,2143-03-18 13:01:00,NaN,EMERGENCY,EMERGENCY ROOM ADMIT,REHAB/DISTINCT PART HOSP,Medicaid,ENGL,OTHER,...,0,1,0,1,0,0,1,0,0,27
2,142768,2164-11-18 07:00:00,2164-11-20 14:30:00,NaN,EMERGENCY,TRANSFER FROM HOSP/EXTRAM,HOME,Private,NaN,CATHOLIC,...,0,0,0,0,0,0,0,0,0,45
3,158569,2142-04-01 12:34:00,2142-04-08 14:46:00,NaN,URGENT,TRANSFER FROM HOSP/EXTRAM,HOME HEALTH CARE,Medicare,NaN,UNOBTAINABLE,...,1,0,0,0,0,0,0,0,0,66
4,170324,2156-03-31 16:11:00,2156-04-29 15:05:00,NaN,EMERGENCY,EMERGENCY ROOM ADMIT,HOME,Government,ENGL,CATHOLIC,...,0,0,0,0,0,0,1,1,0,39


* Tranfer on static_MV

In [36]:
final_formatted_static_MV[["adm_year"]] = final_formatted_static_MV[["admittime"]].apply((lambda x: datetime.datetime.strptime(x["admittime"], "%Y-%m-%d %H:%M:%S").year), axis=1)
final_formatted_static_MV[["dob_year"]] = final_formatted_static_MV[["dob"]].apply((lambda x: datetime.datetime.strptime(x["dob"], "%Y-%m-%d %H:%M:%S").year), axis=1)
final_formatted_static_MV[["age"]] = final_formatted_static_MV["adm_year"] - final_formatted_static_MV["dob_year"]
final_formatted_static_MV.drop(columns=["adm_year", "dob_year"], inplace=True)
final_formatted_static_MV.head()

,hadm_id,admittime,dischtime,deathtime,admission_type,admission_location,discharge_location,insurance,language,religion,...,icd9_12,icd9_13,icd9_14,icd9_15,icd9_16,icd9_17,icd9_18,icd9_19,icd9_20,age
0,165660,2134-05-10 11:30:00,2134-05-20 13:16:00,NaN,ELECTIVE,PHYS REFERRAL/NORMAL DELI,LONG TERM CARE HOSPITAL,Medicare,ENGL,NOT SPECIFIED,...,0,0,0,0,0,0,1,1,0,73
1,126055,2141-10-13 23:10:00,2141-11-03 18:45:00,NaN,EMERGENCY,EMERGENCY ROOM ADMIT,HOME,Medicaid,ENGL,NOT SPECIFIED,...,0,1,0,1,1,0,0,1,0,24
2,136572,2142-06-18 03:11:00,2142-06-20 16:34:00,NaN,EMERGENCY,EMERGENCY ROOM ADMIT,HOME HEALTH CARE,Medicaid,ENGL,NOT SPECIFIED,...,0,1,0,1,0,0,0,1,0,25
3,137510,2142-04-20 13:39:00,2142-04-22 15:30:00,NaN,EMERGENCY,EMERGENCY ROOM ADMIT,HOME HEALTH CARE,Medicaid,ENGL,NOT SPECIFIED,...,0,1,0,1,0,0,0,1,0,25
4,158943,2142-03-21 09:25:00,2142-03-23 18:11:00,NaN,EMERGENCY,EMERGENCY ROOM ADMIT,HOME,Medicaid,ENGL,NOT SPECIFIED,...,0,1,0,1,1,0,1,1,1,25


## d) One-hot encoding other categorical variables

* Transfer on mimic_static_CV

In [37]:
final_formatted_static_CV.columns

Index(['hadm_id', 'admittime', 'dischtime', 'deathtime', 'admission_type',
       'admission_location', 'discharge_location', 'insurance', 'language',
       'religion', 'marital_status', 'ethnicity', 'gender', 'dob', 'dod',
       'icd9_1', 'icd9_2', 'icd9_3', 'icd9_4', 'icd9_5', 'icd9_6', 'icd9_7',
       'icd9_8', 'icd9_9', 'icd9_10', 'icd9_11', 'icd9_12', 'icd9_13',
       'icd9_14', 'icd9_15', 'icd9_16', 'icd9_17', 'icd9_18', 'icd9_19',
       'icd9_20', 'age'],
      dtype='object')

In [38]:
# one-hot encoding "insurance"
insurance_dummy = pd.get_dummies(final_formatted_static_CV.insurance)
final_formatted_static_CV = pd.concat([final_formatted_static_CV,insurance_dummy],axis=1) 

# one-hot encoding "admission_type"
adm_type_dummy = pd.get_dummies(final_formatted_static_CV.admission_type)
final_formatted_static_CV = pd.concat([final_formatted_static_CV, adm_type_dummy], axis=1)

# one-hot encoding "gender"
gender_dummy = pd.get_dummies(final_formatted_static_CV.gender)
final_formatted_static_CV = pd.concat([final_formatted_static_CV, gender_dummy], axis=1)

# drop the original columns where the dummy columns generated
final_formatted_static_CV.drop(['insurance', 'admission_type', 'gender'], axis=1, inplace=True)
# new_mimic_static_formatted is the final form of each admission's static variable
final_formatted_static_CV.head()

,hadm_id,admittime,dischtime,deathtime,admission_location,discharge_location,language,religion,marital_status,ethnicity,...,Government,Medicaid,Medicare,Private,Self Pay,ELECTIVE,EMERGENCY,URGENT,F,M
0,141647,2183-08-20 17:25:00,2183-08-24 15:28:00,NaN,CLINIC REFERRAL/PREMATURE,HOME,NaN,CATHOLIC,SINGLE,WHITE,...,0,0,0,1,0,0,1,0,1,0
1,172056,2143-03-03 09:25:00,2143-03-18 13:01:00,NaN,EMERGENCY ROOM ADMIT,REHAB/DISTINCT PART HOSP,ENGL,OTHER,SINGLE,BLACK/AFRICAN AMERICAN,...,0,1,0,0,0,0,1,0,1,0
2,142768,2164-11-18 07:00:00,2164-11-20 14:30:00,NaN,TRANSFER FROM HOSP/EXTRAM,HOME,NaN,CATHOLIC,MARRIED,UNKNOWN/NOT SPECIFIED,...,0,0,0,1,0,0,1,0,0,1
3,158569,2142-04-01 12:34:00,2142-04-08 14:46:00,NaN,TRANSFER FROM HOSP/EXTRAM,HOME HEALTH CARE,NaN,UNOBTAINABLE,MARRIED,UNKNOWN/NOT SPECIFIED,...,0,0,1,0,0,0,0,1,1,0
4,170324,2156-03-31 16:11:00,2156-04-29 15:05:00,NaN,EMERGENCY ROOM ADMIT,HOME,ENGL,CATHOLIC,MARRIED,WHITE,...,1,0,0,0,0,0,1,0,0,1


* Transfer on mimic_static_MV

In [39]:
final_formatted_static_MV.columns

Index(['hadm_id', 'admittime', 'dischtime', 'deathtime', 'admission_type',
       'admission_location', 'discharge_location', 'insurance', 'language',
       'religion', 'marital_status', 'ethnicity', 'gender', 'dob', 'dod',
       'icd9_1', 'icd9_2', 'icd9_3', 'icd9_4', 'icd9_5', 'icd9_6', 'icd9_7',
       'icd9_8', 'icd9_9', 'icd9_10', 'icd9_11', 'icd9_12', 'icd9_13',
       'icd9_14', 'icd9_15', 'icd9_16', 'icd9_17', 'icd9_18', 'icd9_19',
       'icd9_20', 'age'],
      dtype='object')

In [40]:
# one-hot encoding "insurance"
insurance_dummy = pd.get_dummies(final_formatted_static_MV.insurance)
final_formatted_static_MV = pd.concat([final_formatted_static_MV,insurance_dummy],axis=1) 

# one-hot encoding "admission_type"
adm_type_dummy = pd.get_dummies(final_formatted_static_MV.admission_type)
final_formatted_static_MV = pd.concat([final_formatted_static_MV, adm_type_dummy], axis=1)

# one-hot encoding "gender"
gender_dummy = pd.get_dummies(final_formatted_static_MV.gender)
final_formatted_static_MV = pd.concat([final_formatted_static_MV, gender_dummy], axis=1)

# drop the original columns where the dummy columns generated
final_formatted_static_MV.drop(['insurance', 'admission_type', 'gender'], axis=1, inplace=True)
# new_mimic_static_formatted is the final form of each admission's static variable
with pd.option_context('display.max_rows', None, 'display.max_columns', None): 
    display(final_formatted_static_MV.head())


,hadm_id,admittime,dischtime,deathtime,admission_location,discharge_location,language,religion,marital_status,ethnicity,dob,dod,icd9_1,icd9_2,icd9_3,icd9_4,icd9_5,icd9_6,icd9_7,icd9_8,icd9_9,icd9_10,icd9_11,icd9_12,icd9_13,icd9_14,icd9_15,icd9_16,icd9_17,icd9_18,icd9_19,icd9_20,age,Government,Medicaid,Medicare,Private,Self Pay,ELECTIVE,EMERGENCY,URGENT,F,M
0,165660,2134-05-10 11:30:00,2134-05-20 13:16:00,NaN,PHYS REFERRAL/NORMAL DELI,LONG TERM CARE HOSPITAL,ENGL,NOT SPECIFIED,MARRIED,WHITE,2061-08-17 00:00:00,NaN,0,0,0,0,1,0,1,1,1,1,0,0,0,0,0,0,0,1,1,0,73,0,0,1,0,0,1,0,0,0,1
1,126055,2141-10-13 23:10:00,2141-11-03 18:45:00,NaN,EMERGENCY ROOM ADMIT,HOME,ENGL,NOT SPECIFIED,SINGLE,BLACK/AFRICAN AMERICAN,2117-08-07 00:00:00,2142-08-30 00:00:00,0,0,1,1,0,1,1,1,1,1,0,0,1,0,1,1,0,0,1,0,24,0,1,0,0,0,0,1,0,1,0
2,136572,2142-06-18 03:11:00,2142-06-20 16:34:00,NaN,EMERGENCY ROOM ADMIT,HOME HEALTH CARE,ENGL,NOT SPECIFIED,SINGLE,BLACK/AFRICAN AMERICAN,2117-08-07 00:00:00,2142-08-30 00:00:00,0,0,1,1,1,1,1,0,1,1,0,0,1,0,1,0,0,0,1,0,25,0,1,0,0,0,0,1,0,1,0
3,137510,2142-04-20 13:39:00,2142-04-22 15:30:00,NaN,EMERGENCY ROOM ADMIT,HOME HEALTH CARE,ENGL,NOT SPECIFIED,SINGLE,BLACK/AFRICAN AMERICAN,2117-08-07 00:00:00,2142-08-30 00:00:00,0,0,1,1,0,1,1,0,0,1,0,0,1,0,1,0,0,0,1,0,25,0,1,0,0,0,0,1,0,1,0
4,158943,2142-03-21 09:25:00,2142-03-23 18:11:00,NaN,EMERGENCY ROOM ADMIT,HOME,ENGL,NOT SPECIFIED,SINGLE,BLACK/AFRICAN AMERICAN,2117-08-07 00:00:00,2142-08-30 00:00:00,1,0,1,1,0,1,1,0,0,1,0,0,1,0,1,1,0,1,1,1,25,0,1,0,0,0,0,1,0,1,0


## Totally unique num of adm_id: 26066(CV) & 21018(MV), combine all the dummy variables whose value_counts are less than 10% of 26066(CV)/21018(MV): 2606(CV)/2102(MV) (Only a rough bound)

## h) **one-hot encoding categorical variable: "admission_location"**
 

* final_formatted_static_MV

In [42]:
final_formatted_static_MV["admission_location"].value_counts(dropna=False)

CLINIC REFERRAL/PREMATURE    8188
EMERGENCY ROOM ADMIT         6356
PHYS REFERRAL/NORMAL DELI    3492
TRANSFER FROM HOSP/EXTRAM    2876
TRANSFER FROM SKILLED NUR      68
TRANSFER FROM OTHER HEALT      35
TRSF WITHIN THIS FACILITY       3
Name: admission_location, dtype: int64

In [46]:
combined_adm_loc_values = ["TRANSFER FROM HOSP/EXTRAM", "TRANSFER FROM SKILLED NUR", 
                           "TRANSFER FROM OTHER HEALT", "TRSF WITHIN THIS FACILITY"]
final_formatted_static_MV.loc[final_formatted_static_MV["admission_location"].isin(combined_adm_loc_values), 
                             "admission_location"] = "Other_Adm_Location"
final_formatted_static_MV[final_formatted_static_MV["admission_location"]=="Other_Adm_Location"]

,hadm_id,admittime,dischtime,deathtime,admission_location,discharge_location,language,religion,marital_status,ethnicity,...,Government,Medicaid,Medicare,Private,Self Pay,ELECTIVE,EMERGENCY,URGENT,F,M
36,157999,2113-09-29 21:06:00,2113-10-13 18:06:00,NaN,Other_Adm_Location,LONG TERM CARE HOSPITAL,ENGL,CATHOLIC,MARRIED,WHITE,...,0,0,1,0,0,0,1,0,1,0
60,105852,2169-11-08 18:28:00,2169-11-09 14:40:00,NaN,Other_Adm_Location,SHORT TERM HOSPITAL,PTUN,OTHER,WIDOWED,WHITE,...,0,0,1,0,0,0,1,0,0,1
89,106870,2206-06-18 17:25:00,2206-07-03 13:45:00,NaN,Other_Adm_Location,REHAB/DISTINCT PART HOSP,ENGL,CATHOLIC,MARRIED,WHITE,...,0,0,1,0,0,0,1,0,0,1
100,103203,2149-10-07 20:06:00,2149-10-20 15:30:00,NaN,Other_Adm_Location,HOME HEALTH CARE,ENGL,CATHOLIC,SINGLE,WHITE,...,0,0,1,0,0,0,1,0,1,0
101,138805,2148-09-25 13:56:00,2148-10-02 15:00:00,NaN,Other_Adm_Location,HOME HEALTH CARE,ENGL,CATHOLIC,SINGLE,WHITE,...,0,0,1,0,0,0,1,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20995,170022,2200-01-12 20:26:00,2200-01-20 16:46:00,NaN,Other_Adm_Location,HOME,SPAN,CATHOLIC,SINGLE,HISPANIC OR LATINO,...,0,0,0,1,0,0,0,1,0,1
21000,121397,2177-03-10 16:59:00,2177-03-20 18:17:00,NaN,Other_Adm_Location,REHAB/DISTINCT PART HOSP,ENGL,NOT SPECIFIED,MARRIED,UNKNOWN/NOT SPECIFIED,...,0,0,1,0,0,0,1,0,0,1
21002,135263,2141-12-31 11:35:00,2142-01-29 17:10:00,2142-01-29 17:10:00,Other_Adm_Location,DEAD/EXPIRED,NaN,UNOBTAINABLE,WIDOWED,PATIENT DECLINED TO ANSWER,...,0,0,1,0,0,0,1,0,1,0
21004,175780,2105-02-15 18:00:00,2105-03-03 21:00:00,NaN,Other_Adm_Location,HOME,ENGL,UNOBTAINABLE,MARRIED,WHITE,...,0,0,0,1,0,0,1,0,1,0


In [48]:
adm_loc_dummy = pd.get_dummies(final_formatted_static_MV.admission_location)
final_formatted_static_MV = pd.concat([final_formatted_static_MV, adm_loc_dummy],axis=1)
final_formatted_static_MV.drop(columns="admission_location", axis=1, inplace=True)
# new_mimic_static_formatted is the final form of each admission's static variable
with pd.option_context('display.max_rows', None, 'display.max_columns', None): 
    display(final_formatted_static_MV.head())


,hadm_id,admittime,dischtime,deathtime,discharge_location,language,religion,marital_status,ethnicity,dob,dod,icd9_1,icd9_2,icd9_3,icd9_4,icd9_5,icd9_6,icd9_7,icd9_8,icd9_9,icd9_10,icd9_11,icd9_12,icd9_13,icd9_14,icd9_15,icd9_16,icd9_17,icd9_18,icd9_19,icd9_20,age,Government,Medicaid,Medicare,Private,Self Pay,ELECTIVE,EMERGENCY,URGENT,F,M,CLINIC REFERRAL/PREMATURE,EMERGENCY ROOM ADMIT,Other_Adm_Location,PHYS REFERRAL/NORMAL DELI
0,165660,2134-05-10 11:30:00,2134-05-20 13:16:00,NaN,LONG TERM CARE HOSPITAL,ENGL,NOT SPECIFIED,MARRIED,WHITE,2061-08-17 00:00:00,NaN,0,0,0,0,1,0,1,1,1,1,0,0,0,0,0,0,0,1,1,0,73,0,0,1,0,0,1,0,0,0,1,0,0,0,1
1,126055,2141-10-13 23:10:00,2141-11-03 18:45:00,NaN,HOME,ENGL,NOT SPECIFIED,SINGLE,BLACK/AFRICAN AMERICAN,2117-08-07 00:00:00,2142-08-30 00:00:00,0,0,1,1,0,1,1,1,1,1,0,0,1,0,1,1,0,0,1,0,24,0,1,0,0,0,0,1,0,1,0,0,1,0,0
2,136572,2142-06-18 03:11:00,2142-06-20 16:34:00,NaN,HOME HEALTH CARE,ENGL,NOT SPECIFIED,SINGLE,BLACK/AFRICAN AMERICAN,2117-08-07 00:00:00,2142-08-30 00:00:00,0,0,1,1,1,1,1,0,1,1,0,0,1,0,1,0,0,0,1,0,25,0,1,0,0,0,0,1,0,1,0,0,1,0,0
3,137510,2142-04-20 13:39:00,2142-04-22 15:30:00,NaN,HOME HEALTH CARE,ENGL,NOT SPECIFIED,SINGLE,BLACK/AFRICAN AMERICAN,2117-08-07 00:00:00,2142-08-30 00:00:00,0,0,1,1,0,1,1,0,0,1,0,0,1,0,1,0,0,0,1,0,25,0,1,0,0,0,0,1,0,1,0,0,1,0,0
4,158943,2142-03-21 09:25:00,2142-03-23 18:11:00,NaN,HOME,ENGL,NOT SPECIFIED,SINGLE,BLACK/AFRICAN AMERICAN,2117-08-07 00:00:00,2142-08-30 00:00:00,1,0,1,1,0,1,1,0,0,1,0,0,1,0,1,1,0,1,1,1,25,0,1,0,0,0,0,1,0,1,0,0,1,0,0


* final_formatted_static_CV

In [49]:
final_formatted_static_CV["admission_location"].value_counts(dropna=False)

EMERGENCY ROOM ADMIT         14659
TRANSFER FROM HOSP/EXTRAM     5099
PHYS REFERRAL/NORMAL DELI     4834
CLINIC REFERRAL/PREMATURE     1264
TRANSFER FROM SKILLED NUR      174
TRANSFER FROM OTHER HEALT       31
** INFO NOT AVAILABLE **         4
HMO REFERRAL/SICK                1
Name: admission_location, dtype: int64

In [50]:
combined_adm_loc_values_cv = ["CLINIC REFERRAL/PREMATURE", "TRANSFER FROM SKILLED NUR", "TRANSFER FROM OTHER HEALT",
                             "** INFO NOT AVAILABLE **", "HMO REFERRAL/SICK"]
final_formatted_static_CV.loc[final_formatted_static_CV["admission_location"].isin(combined_adm_loc_values_cv), "admission_location"] = "Other_Adm_Location"

In [51]:
adm_loc_dummy_cv = pd.get_dummies(final_formatted_static_CV.admission_location)
final_formatted_static_CV = pd.concat([final_formatted_static_CV, adm_loc_dummy_cv],axis=1)
final_formatted_static_CV.drop(columns="admission_location", axis=1, inplace=True)
# new_mimic_static_formatted is the final form of each admission's static variable
with pd.option_context('display.max_rows', None, 'display.max_columns', None): 
    display(final_formatted_static_CV.head())


,hadm_id,admittime,dischtime,deathtime,discharge_location,language,religion,marital_status,ethnicity,dob,dod,icd9_1,icd9_2,icd9_3,icd9_4,icd9_5,icd9_6,icd9_7,icd9_8,icd9_9,icd9_10,icd9_11,icd9_12,icd9_13,icd9_14,icd9_15,icd9_16,icd9_17,icd9_18,icd9_19,icd9_20,age,Government,Medicaid,Medicare,Private,Self Pay,ELECTIVE,EMERGENCY,URGENT,F,M,EMERGENCY ROOM ADMIT,Other_Adm_Location,PHYS REFERRAL/NORMAL DELI,TRANSFER FROM HOSP/EXTRAM
0,141647,2183-08-20 17:25:00,2183-08-24 15:28:00,NaN,HOME,NaN,CATHOLIC,SINGLE,WHITE,2135-02-01 00:00:00,2184-02-22 00:00:00,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,48,0,0,0,1,0,0,1,0,1,0,0,1,0,0
1,172056,2143-03-03 09:25:00,2143-03-18 13:01:00,NaN,REHAB/DISTINCT PART HOSP,ENGL,OTHER,SINGLE,BLACK/AFRICAN AMERICAN,2116-06-27 00:00:00,NaN,1,0,0,0,1,0,1,1,0,0,0,0,1,0,1,0,0,1,0,0,27,0,1,0,0,0,0,1,0,1,0,1,0,0,0
2,142768,2164-11-18 07:00:00,2164-11-20 14:30:00,NaN,HOME,NaN,CATHOLIC,MARRIED,UNKNOWN/NOT SPECIFIED,2119-05-12 00:00:00,NaN,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,45,0,0,0,1,0,0,1,0,0,1,0,0,0,1
3,158569,2142-04-01 12:34:00,2142-04-08 14:46:00,NaN,HOME HEALTH CARE,NaN,UNOBTAINABLE,MARRIED,UNKNOWN/NOT SPECIFIED,2076-09-15 00:00:00,2142-11-20 00:00:00,0,0,1,0,0,0,1,0,1,0,0,1,0,0,0,0,0,0,0,0,66,0,0,1,0,0,0,0,1,1,0,0,0,0,1
4,170324,2156-03-31 16:11:00,2156-04-29 15:05:00,NaN,HOME,ENGL,CATHOLIC,MARRIED,WHITE,2117-10-02 00:00:00,NaN,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,1,1,0,39,1,0,0,0,0,0,1,0,0,1,1,0,0,0


## i) **one-hot encoding "discharge_location" categorical static variable**


* apply on final_formatted_static_MV (combine_bound: 2102)

In [55]:
discharge_loc_value_counts_MV = final_formatted_static_MV["discharge_location"].value_counts(dropna=False).rename_axis('unique_values').reset_index(name='counts')
discharge_loc_value_counts_MV

,unique_values,counts
0,HOME HEALTH CARE,5433
1,HOME,5410
2,SNF,3687
3,DEAD/EXPIRED,1832
4,LONG TERM CARE HOSPITAL,1831
5,REHAB/DISTINCT PART HOSP,1810
6,DISCH-TRAN TO PSYCH HOSP,257
7,HOSPICE-HOME,218
8,SHORT TERM HOSPITAL,211
9,LEFT AGAINST MEDICAL ADVI,158


In [56]:
combined_discharge_loc_values_list = list(discharge_loc_value_counts_MV[discharge_loc_value_counts_MV["counts"]<2102]["unique_values"])
combined_discharge_loc_values_list

['DEAD/EXPIRED',
 'LONG TERM CARE HOSPITAL',
 'REHAB/DISTINCT PART HOSP',
 'DISCH-TRAN TO PSYCH HOSP',
 'HOSPICE-HOME',
 'SHORT TERM HOSPITAL',
 'LEFT AGAINST MEDICAL ADVI',
 'HOSPICE-MEDICAL FACILITY',
 'OTHER FACILITY',
 'ICF',
 'DISC-TRAN TO FEDERAL HC']

In [57]:
final_formatted_static_MV.loc[final_formatted_static_MV["discharge_location"].isin(combined_discharge_loc_values_list), 
                              "discharge_location"] = "Other_Discharge_Location"

In [58]:
disc_loc_dummy_mv = pd.get_dummies(final_formatted_static_MV.discharge_location)
final_formatted_static_MV = pd.concat([final_formatted_static_MV, disc_loc_dummy_mv],axis=1)
final_formatted_static_MV.drop(columns="discharge_location", axis=1, inplace=True)
# new_mimic_static_formatted is the final form of each admission's static variable
with pd.option_context('display.max_rows', None, 'display.max_columns', None): 
    display(final_formatted_static_MV.head())

,hadm_id,admittime,dischtime,deathtime,language,religion,marital_status,ethnicity,dob,dod,icd9_1,icd9_2,icd9_3,icd9_4,icd9_5,icd9_6,icd9_7,icd9_8,icd9_9,icd9_10,icd9_11,icd9_12,icd9_13,icd9_14,icd9_15,icd9_16,icd9_17,icd9_18,icd9_19,icd9_20,age,Government,Medicaid,Medicare,Private,Self Pay,ELECTIVE,EMERGENCY,URGENT,F,M,CLINIC REFERRAL/PREMATURE,EMERGENCY ROOM ADMIT,Other_Adm_Location,PHYS REFERRAL/NORMAL DELI,HOME,HOME HEALTH CARE,Other_Discharge_Location,SNF
0,165660,2134-05-10 11:30:00,2134-05-20 13:16:00,NaN,ENGL,NOT SPECIFIED,MARRIED,WHITE,2061-08-17 00:00:00,NaN,0,0,0,0,1,0,1,1,1,1,0,0,0,0,0,0,0,1,1,0,73,0,0,1,0,0,1,0,0,0,1,0,0,0,1,0,0,1,0
1,126055,2141-10-13 23:10:00,2141-11-03 18:45:00,NaN,ENGL,NOT SPECIFIED,SINGLE,BLACK/AFRICAN AMERICAN,2117-08-07 00:00:00,2142-08-30 00:00:00,0,0,1,1,0,1,1,1,1,1,0,0,1,0,1,1,0,0,1,0,24,0,1,0,0,0,0,1,0,1,0,0,1,0,0,1,0,0,0
2,136572,2142-06-18 03:11:00,2142-06-20 16:34:00,NaN,ENGL,NOT SPECIFIED,SINGLE,BLACK/AFRICAN AMERICAN,2117-08-07 00:00:00,2142-08-30 00:00:00,0,0,1,1,1,1,1,0,1,1,0,0,1,0,1,0,0,0,1,0,25,0,1,0,0,0,0,1,0,1,0,0,1,0,0,0,1,0,0
3,137510,2142-04-20 13:39:00,2142-04-22 15:30:00,NaN,ENGL,NOT SPECIFIED,SINGLE,BLACK/AFRICAN AMERICAN,2117-08-07 00:00:00,2142-08-30 00:00:00,0,0,1,1,0,1,1,0,0,1,0,0,1,0,1,0,0,0,1,0,25,0,1,0,0,0,0,1,0,1,0,0,1,0,0,0,1,0,0
4,158943,2142-03-21 09:25:00,2142-03-23 18:11:00,NaN,ENGL,NOT SPECIFIED,SINGLE,BLACK/AFRICAN AMERICAN,2117-08-07 00:00:00,2142-08-30 00:00:00,1,0,1,1,0,1,1,0,0,1,0,0,1,0,1,1,0,1,1,1,25,0,1,0,0,0,0,1,0,1,0,0,1,0,0,1,0,0,0


* apply on final_formatted_static_CV (combine_bound: 2606)

In [59]:
discharge_loc_value_counts_CV = final_formatted_static_CV["discharge_location"].value_counts(dropna=False).rename_axis('unique_values').reset_index(name='counts')
discharge_loc_value_counts_CV

,unique_values,counts
0,HOME,6933
1,HOME HEALTH CARE,6568
2,REHAB/DISTINCT PART HOSP,4323
3,SNF,3628
4,DEAD/EXPIRED,2772
5,DISC-TRAN CANCER/CHLDRN H,553
6,LONG TERM CARE HOSPITAL,382
7,SHORT TERM HOSPITAL,253
8,DISCH-TRAN TO PSYCH HOSP,171
9,LEFT AGAINST MEDICAL ADVI,170


In [60]:
combined_discharge_loc_values_list_CV = list(discharge_loc_value_counts_CV[discharge_loc_value_counts_CV["counts"]<2800]["unique_values"])
final_formatted_static_CV.loc[final_formatted_static_CV["discharge_location"].isin(combined_discharge_loc_values_list_CV), 
                              "discharge_location"] = "Other_Discharge_Location"
disc_loc_dummy_CV = pd.get_dummies(final_formatted_static_CV.discharge_location)
final_formatted_static_CV = pd.concat([final_formatted_static_CV, disc_loc_dummy_CV],axis=1)
final_formatted_static_CV.drop(columns="discharge_location", axis=1, inplace=True)
# new_mimic_static_formatted is the final form of each admission's static variable
with pd.option_context('display.max_rows', None, 'display.max_columns', None): 
    display(final_formatted_static_CV.head())

,hadm_id,admittime,dischtime,deathtime,language,religion,marital_status,ethnicity,dob,dod,icd9_1,icd9_2,icd9_3,icd9_4,icd9_5,icd9_6,icd9_7,icd9_8,icd9_9,icd9_10,icd9_11,icd9_12,icd9_13,icd9_14,icd9_15,icd9_16,icd9_17,icd9_18,icd9_19,icd9_20,age,Government,Medicaid,Medicare,Private,Self Pay,ELECTIVE,EMERGENCY,URGENT,F,M,EMERGENCY ROOM ADMIT,Other_Adm_Location,PHYS REFERRAL/NORMAL DELI,TRANSFER FROM HOSP/EXTRAM,HOME,HOME HEALTH CARE,Other_Discharge_Location,REHAB/DISTINCT PART HOSP,SNF
0,141647,2183-08-20 17:25:00,2183-08-24 15:28:00,NaN,NaN,CATHOLIC,SINGLE,WHITE,2135-02-01 00:00:00,2184-02-22 00:00:00,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,48,0,0,0,1,0,0,1,0,1,0,0,1,0,0,1,0,0,0,0
1,172056,2143-03-03 09:25:00,2143-03-18 13:01:00,NaN,ENGL,OTHER,SINGLE,BLACK/AFRICAN AMERICAN,2116-06-27 00:00:00,NaN,1,0,0,0,1,0,1,1,0,0,0,0,1,0,1,0,0,1,0,0,27,0,1,0,0,0,0,1,0,1,0,1,0,0,0,0,0,0,1,0
2,142768,2164-11-18 07:00:00,2164-11-20 14:30:00,NaN,NaN,CATHOLIC,MARRIED,UNKNOWN/NOT SPECIFIED,2119-05-12 00:00:00,NaN,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,45,0,0,0,1,0,0,1,0,0,1,0,0,0,1,1,0,0,0,0
3,158569,2142-04-01 12:34:00,2142-04-08 14:46:00,NaN,NaN,UNOBTAINABLE,MARRIED,UNKNOWN/NOT SPECIFIED,2076-09-15 00:00:00,2142-11-20 00:00:00,0,0,1,0,0,0,1,0,1,0,0,1,0,0,0,0,0,0,0,0,66,0,0,1,0,0,0,0,1,1,0,0,0,0,1,0,1,0,0,0
4,170324,2156-03-31 16:11:00,2156-04-29 15:05:00,NaN,ENGL,CATHOLIC,MARRIED,WHITE,2117-10-02 00:00:00,NaN,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,1,1,0,39,1,0,0,0,0,0,1,0,0,1,1,0,0,0,1,0,0,0,0


## j) one-hot encoding categorical variable: "language"

* mv

In [64]:
lang_value_counts_mv = final_formatted_static_MV["language"].value_counts(dropna=False).rename_axis("unique_values").reset_index(name="counts")
# with pd.option_context('display.max_rows', None, 'display.max_columns', None): 
#     display(lang_value_counts_mv)

In [66]:
combined_lang_values_mv = list(lang_value_counts_mv[lang_value_counts_mv["counts"] < 2102]["unique_values"])
# combined_lang_values_mv

In [67]:
final_formatted_static_MV.loc[final_formatted_static_MV["language"].isin(combined_lang_values_mv), "language"] = "Other_Language"

In [69]:
lang_dummies_mv = pd.get_dummies(final_formatted_static_MV.language)
final_formatted_static_MV = pd.concat([final_formatted_static_MV, lang_dummies_mv], axis=1)
final_formatted_static_MV.drop(columns="language", axis=1, inplace=True)
with pd.option_context('display.max_rows', None, 'display.max_columns', None): 
    display(final_formatted_static_MV.head())

,hadm_id,admittime,dischtime,deathtime,religion,marital_status,ethnicity,dob,dod,icd9_1,icd9_2,icd9_3,icd9_4,icd9_5,icd9_6,icd9_7,icd9_8,icd9_9,icd9_10,icd9_11,icd9_12,icd9_13,icd9_14,icd9_15,icd9_16,icd9_17,icd9_18,icd9_19,icd9_20,age,Government,Medicaid,Medicare,Private,Self Pay,ELECTIVE,EMERGENCY,URGENT,F,M,CLINIC REFERRAL/PREMATURE,EMERGENCY ROOM ADMIT,Other_Adm_Location,PHYS REFERRAL/NORMAL DELI,HOME,HOME HEALTH CARE,Other_Discharge_Location,SNF,ENGL,Other_Language
0,165660,2134-05-10 11:30:00,2134-05-20 13:16:00,NaN,NOT SPECIFIED,MARRIED,WHITE,2061-08-17 00:00:00,NaN,0,0,0,0,1,0,1,1,1,1,0,0,0,0,0,0,0,1,1,0,73,0,0,1,0,0,1,0,0,0,1,0,0,0,1,0,0,1,0,1,0
1,126055,2141-10-13 23:10:00,2141-11-03 18:45:00,NaN,NOT SPECIFIED,SINGLE,BLACK/AFRICAN AMERICAN,2117-08-07 00:00:00,2142-08-30 00:00:00,0,0,1,1,0,1,1,1,1,1,0,0,1,0,1,1,0,0,1,0,24,0,1,0,0,0,0,1,0,1,0,0,1,0,0,1,0,0,0,1,0
2,136572,2142-06-18 03:11:00,2142-06-20 16:34:00,NaN,NOT SPECIFIED,SINGLE,BLACK/AFRICAN AMERICAN,2117-08-07 00:00:00,2142-08-30 00:00:00,0,0,1,1,1,1,1,0,1,1,0,0,1,0,1,0,0,0,1,0,25,0,1,0,0,0,0,1,0,1,0,0,1,0,0,0,1,0,0,1,0
3,137510,2142-04-20 13:39:00,2142-04-22 15:30:00,NaN,NOT SPECIFIED,SINGLE,BLACK/AFRICAN AMERICAN,2117-08-07 00:00:00,2142-08-30 00:00:00,0,0,1,1,0,1,1,0,0,1,0,0,1,0,1,0,0,0,1,0,25,0,1,0,0,0,0,1,0,1,0,0,1,0,0,0,1,0,0,1,0
4,158943,2142-03-21 09:25:00,2142-03-23 18:11:00,NaN,NOT SPECIFIED,SINGLE,BLACK/AFRICAN AMERICAN,2117-08-07 00:00:00,2142-08-30 00:00:00,1,0,1,1,0,1,1,0,0,1,0,0,1,0,1,1,0,1,1,1,25,0,1,0,0,0,0,1,0,1,0,0,1,0,0,1,0,0,0,1,0


* cv

In [78]:
lang_value_counts_cv = final_formatted_static_CV["language"].value_counts(dropna=False).rename_axis("unique_values").reset_index(name="counts")
lang_value_counts_cv.loc[0, "unique_values"] = "Lan_Missing"
# with pd.option_context('display.max_rows', None, 'display.max_columns', None): 
#     display(lang_value_counts_cv)
combined_lang_values_cv = list(lang_value_counts_cv[lang_value_counts_cv["counts"] < 2606]["unique_values"])
final_formatted_static_CV.loc[final_formatted_static_CV["language"].isin(combined_lang_values_cv), "language"] = "Other_Language"
final_formatted_static_CV["language"].value_counts(dropna=False)

NaN               15885
ENGL               8058
Other_Language     2123
Name: language, dtype: int64

In [85]:
# transfer the "NaN" in final_formatted_static_CV["language"] to "Lan_Missing"
final_formatted_static_CV.loc[final_formatted_static_CV["language"].isnull(), "language"] = "Lan_Missing"
final_formatted_static_CV["language"].value_counts(dropna=False)

Lan_Missing       15885
ENGL               8058
Other_Language     2123
Name: language, dtype: int64

In [86]:
lang_dummies_cv = pd.get_dummies(final_formatted_static_CV.language)
final_formatted_static_CV = pd.concat([final_formatted_static_CV, lang_dummies_cv], axis=1)
final_formatted_static_CV.drop(columns="language", axis=1, inplace=True)
with pd.option_context('display.max_rows', None, 'display.max_columns', None): 
    display(final_formatted_static_CV.head())

,hadm_id,admittime,dischtime,deathtime,religion,marital_status,ethnicity,dob,dod,icd9_1,icd9_2,icd9_3,icd9_4,icd9_5,icd9_6,icd9_7,icd9_8,icd9_9,icd9_10,icd9_11,icd9_12,icd9_13,icd9_14,icd9_15,icd9_16,icd9_17,icd9_18,icd9_19,icd9_20,age,Government,Medicaid,Medicare,Private,Self Pay,ELECTIVE,EMERGENCY,URGENT,F,M,EMERGENCY ROOM ADMIT,Other_Adm_Location,PHYS REFERRAL/NORMAL DELI,TRANSFER FROM HOSP/EXTRAM,HOME,HOME HEALTH CARE,Other_Discharge_Location,REHAB/DISTINCT PART HOSP,SNF,ENGL,Lan_Missing,Other_Language
0,141647,2183-08-20 17:25:00,2183-08-24 15:28:00,NaN,CATHOLIC,SINGLE,WHITE,2135-02-01 00:00:00,2184-02-22 00:00:00,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,48,0,0,0,1,0,0,1,0,1,0,0,1,0,0,1,0,0,0,0,0,1,0
1,172056,2143-03-03 09:25:00,2143-03-18 13:01:00,NaN,OTHER,SINGLE,BLACK/AFRICAN AMERICAN,2116-06-27 00:00:00,NaN,1,0,0,0,1,0,1,1,0,0,0,0,1,0,1,0,0,1,0,0,27,0,1,0,0,0,0,1,0,1,0,1,0,0,0,0,0,0,1,0,1,0,0
2,142768,2164-11-18 07:00:00,2164-11-20 14:30:00,NaN,CATHOLIC,MARRIED,UNKNOWN/NOT SPECIFIED,2119-05-12 00:00:00,NaN,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,45,0,0,0,1,0,0,1,0,0,1,0,0,0,1,1,0,0,0,0,0,1,0
3,158569,2142-04-01 12:34:00,2142-04-08 14:46:00,NaN,UNOBTAINABLE,MARRIED,UNKNOWN/NOT SPECIFIED,2076-09-15 00:00:00,2142-11-20 00:00:00,0,0,1,0,0,0,1,0,1,0,0,1,0,0,0,0,0,0,0,0,66,0,0,1,0,0,0,0,1,1,0,0,0,0,1,0,1,0,0,0,0,1,0
4,170324,2156-03-31 16:11:00,2156-04-29 15:05:00,NaN,CATHOLIC,MARRIED,WHITE,2117-10-02 00:00:00,NaN,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,1,1,0,39,1,0,0,0,0,0,1,0,0,1,1,0,0,0,1,0,0,0,0,1,0,0


## k) one-hot encoding categorical variable "religion"

* MV

In [89]:
reli_value_counts_mv = final_formatted_static_MV["religion"].value_counts(dropna=False).rename_axis("unique_values").reset_index(name="counts")
combined_reli_mv = list(reli_value_counts_mv[reli_value_counts_mv["counts"] < 2102]["unique_values"])

In [90]:
final_formatted_static_MV.loc[final_formatted_static_MV["religion"].isin(combined_reli_mv), "religion"] = "Other_Religion"
final_formatted_static_MV["religion"].value_counts()

CATHOLIC             7679
NOT SPECIFIED        5586
Other_Religion       5089
PROTESTANT QUAKER    2664
Name: religion, dtype: int64

In [91]:
final_formatted_static_MV.loc[final_formatted_static_MV["religion"]=="NOT SPECIFIED", "religion"] = "Religion_missing"


TypeError: value_counts() got an unexpected keyword argument 'dropna'

In [92]:
final_formatted_static_MV["religion"].value_counts(dropna=False)

CATHOLIC             7679
Religion_missing     5586
Other_Religion       5089
PROTESTANT QUAKER    2664
Name: religion, dtype: int64

In [93]:
reli_dummy_mv = pd.get_dummies(final_formatted_static_MV.religion)
final_formatted_static_MV = pd.concat([final_formatted_static_MV, reli_dummy_mv], axis=1)
final_formatted_static_MV.drop(columns="religion", axis=1, inplace=True)
with pd.option_context('display.max_rows', None, 'display.max_columns', None): 
    display(final_formatted_static_MV.head())


,hadm_id,admittime,dischtime,deathtime,marital_status,ethnicity,dob,dod,icd9_1,icd9_2,icd9_3,icd9_4,icd9_5,icd9_6,icd9_7,icd9_8,icd9_9,icd9_10,icd9_11,icd9_12,icd9_13,icd9_14,icd9_15,icd9_16,icd9_17,icd9_18,icd9_19,icd9_20,age,Government,Medicaid,Medicare,Private,Self Pay,ELECTIVE,EMERGENCY,URGENT,F,M,CLINIC REFERRAL/PREMATURE,EMERGENCY ROOM ADMIT,Other_Adm_Location,PHYS REFERRAL/NORMAL DELI,HOME,HOME HEALTH CARE,Other_Discharge_Location,SNF,ENGL,Other_Language,CATHOLIC,Other_Religion,PROTESTANT QUAKER,Religion_missing
0,165660,2134-05-10 11:30:00,2134-05-20 13:16:00,NaN,MARRIED,WHITE,2061-08-17 00:00:00,NaN,0,0,0,0,1,0,1,1,1,1,0,0,0,0,0,0,0,1,1,0,73,0,0,1,0,0,1,0,0,0,1,0,0,0,1,0,0,1,0,1,0,0,0,0,1
1,126055,2141-10-13 23:10:00,2141-11-03 18:45:00,NaN,SINGLE,BLACK/AFRICAN AMERICAN,2117-08-07 00:00:00,2142-08-30 00:00:00,0,0,1,1,0,1,1,1,1,1,0,0,1,0,1,1,0,0,1,0,24,0,1,0,0,0,0,1,0,1,0,0,1,0,0,1,0,0,0,1,0,0,0,0,1
2,136572,2142-06-18 03:11:00,2142-06-20 16:34:00,NaN,SINGLE,BLACK/AFRICAN AMERICAN,2117-08-07 00:00:00,2142-08-30 00:00:00,0,0,1,1,1,1,1,0,1,1,0,0,1,0,1,0,0,0,1,0,25,0,1,0,0,0,0,1,0,1,0,0,1,0,0,0,1,0,0,1,0,0,0,0,1
3,137510,2142-04-20 13:39:00,2142-04-22 15:30:00,NaN,SINGLE,BLACK/AFRICAN AMERICAN,2117-08-07 00:00:00,2142-08-30 00:00:00,0,0,1,1,0,1,1,0,0,1,0,0,1,0,1,0,0,0,1,0,25,0,1,0,0,0,0,1,0,1,0,0,1,0,0,0,1,0,0,1,0,0,0,0,1
4,158943,2142-03-21 09:25:00,2142-03-23 18:11:00,NaN,SINGLE,BLACK/AFRICAN AMERICAN,2117-08-07 00:00:00,2142-08-30 00:00:00,1,0,1,1,0,1,1,0,0,1,0,0,1,0,1,1,0,1,1,1,25,0,1,0,0,0,0,1,0,1,0,0,1,0,0,1,0,0,0,1,0,0,0,0,1


* CV

In [94]:
reli_value_counts_cv = final_formatted_static_CV["religion"].value_counts(dropna=False).rename_axis("unique_values").reset_index(name="counts")
reli_value_counts_cv

,unique_values,counts
0,CATHOLIC,9479
1,NOT SPECIFIED,4053
2,UNOBTAINABLE,3441
3,PROTESTANT QUAKER,3396
4,JEWISH,2617
5,OTHER,1538
6,NaN,424
7,EPISCOPALIAN,363
8,GREEK ORTHODOX,201
9,CHRISTIAN SCIENTIST,147


In [95]:
# combine those unique values with less than 2606 value_counts into one unique value "Other_Religion"
# transfer the "religion" with value "NOT SPECIFIED" or "UNOBTAINABLE" into value "Religion_missing"
combined_reli_cv = list(reli_value_counts_cv[reli_value_counts_cv["counts"] < 2606]["unique_values"])
final_formatted_static_CV.loc[final_formatted_static_CV["religion"].isin(combined_reli_cv), "religion"] = "Other_Religion"
final_formatted_static_CV["religion"].value_counts(dropna=False)

CATHOLIC             9479
NOT SPECIFIED        4053
UNOBTAINABLE         3441
PROTESTANT QUAKER    3396
Other_Religion       3080
JEWISH               2617
Name: religion, dtype: int64

In [96]:
final_formatted_static_CV.loc[final_formatted_static_CV["religion"].isin(["NOT SPECIFIED", "UNOBTAINABLE"]), "religion"] = "Religion_missing"
final_formatted_static_CV["religion"].value_counts(dropna=False)

CATHOLIC             9479
Religion_missing     7494
PROTESTANT QUAKER    3396
Other_Religion       3080
JEWISH               2617
Name: religion, dtype: int64

In [97]:
reli_dummy_cv = pd.get_dummies(final_formatted_static_CV.religion)
final_formatted_static_CV = pd.concat([final_formatted_static_CV, reli_dummy_cv], axis=1)
final_formatted_static_CV.drop(columns="religion", axis=1, inplace=True)
with pd.option_context('display.max_rows', None, 'display.max_columns', None): 
    display(final_formatted_static_CV.head())

,hadm_id,admittime,dischtime,deathtime,marital_status,ethnicity,dob,dod,icd9_1,icd9_2,icd9_3,icd9_4,icd9_5,icd9_6,icd9_7,icd9_8,icd9_9,icd9_10,icd9_11,icd9_12,icd9_13,icd9_14,icd9_15,icd9_16,icd9_17,icd9_18,icd9_19,icd9_20,age,Government,Medicaid,Medicare,Private,Self Pay,ELECTIVE,EMERGENCY,URGENT,F,M,EMERGENCY ROOM ADMIT,Other_Adm_Location,PHYS REFERRAL/NORMAL DELI,TRANSFER FROM HOSP/EXTRAM,HOME,HOME HEALTH CARE,Other_Discharge_Location,REHAB/DISTINCT PART HOSP,SNF,ENGL,Lan_Missing,Other_Language,CATHOLIC,JEWISH,Other_Religion,PROTESTANT QUAKER,Religion_missing
0,141647,2183-08-20 17:25:00,2183-08-24 15:28:00,NaN,SINGLE,WHITE,2135-02-01 00:00:00,2184-02-22 00:00:00,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,48,0,0,0,1,0,0,1,0,1,0,0,1,0,0,1,0,0,0,0,0,1,0,1,0,0,0,0
1,172056,2143-03-03 09:25:00,2143-03-18 13:01:00,NaN,SINGLE,BLACK/AFRICAN AMERICAN,2116-06-27 00:00:00,NaN,1,0,0,0,1,0,1,1,0,0,0,0,1,0,1,0,0,1,0,0,27,0,1,0,0,0,0,1,0,1,0,1,0,0,0,0,0,0,1,0,1,0,0,0,0,1,0,0
2,142768,2164-11-18 07:00:00,2164-11-20 14:30:00,NaN,MARRIED,UNKNOWN/NOT SPECIFIED,2119-05-12 00:00:00,NaN,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,45,0,0,0,1,0,0,1,0,0,1,0,0,0,1,1,0,0,0,0,0,1,0,1,0,0,0,0
3,158569,2142-04-01 12:34:00,2142-04-08 14:46:00,NaN,MARRIED,UNKNOWN/NOT SPECIFIED,2076-09-15 00:00:00,2142-11-20 00:00:00,0,0,1,0,0,0,1,0,1,0,0,1,0,0,0,0,0,0,0,0,66,0,0,1,0,0,0,0,1,1,0,0,0,0,1,0,1,0,0,0,0,1,0,0,0,0,0,1
4,170324,2156-03-31 16:11:00,2156-04-29 15:05:00,NaN,MARRIED,WHITE,2117-10-02 00:00:00,NaN,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,1,1,0,39,1,0,0,0,0,0,1,0,0,1,1,0,0,0,1,0,0,0,0,1,0,0,1,0,0,0,0


## l) one-hot encoding categorical variable "marital_status"

* MV

In [98]:
marital_value_counts_MV = final_formatted_static_MV["marital_status"].value_counts(dropna=False).rename_axis("unique_values").reset_index(name="counts")
marital_value_counts_MV

,unique_values,counts
0,MARRIED,9801
1,SINGLE,5946
2,WIDOWED,2797
3,DIVORCED,1404
4,NaN,722
5,SEPARATED,232
6,UNKNOWN (DEFAULT),104
7,LIFE PARTNER,12


In [99]:
combined_marital_mv = list(marital_value_counts_MV[marital_value_counts_MV["counts"] < 2102]["unique_values"])
final_formatted_static_MV.loc[final_formatted_static_MV["marital_status"].isin(combined_marital_mv), "marital_status"] = "Other_Marital_Status"
final_formatted_static_MV["marital_status"].value_counts(dropna=False)

MARRIED                 9801
SINGLE                  5946
WIDOWED                 2797
Other_Marital_Status    2474
Name: marital_status, dtype: int64

In [100]:
marital_dummy_mv = pd.get_dummies(final_formatted_static_MV.marital_status)
final_formatted_static_MV = pd.concat([final_formatted_static_MV, marital_dummy_mv], axis=1)
final_formatted_static_MV.drop(columns="marital_status", inplace=True, axis=1)
with pd.option_context('display.max_rows', None, 'display.max_columns', None): 
    display(final_formatted_static_MV.head())

,hadm_id,admittime,dischtime,deathtime,ethnicity,dob,dod,icd9_1,icd9_2,icd9_3,icd9_4,icd9_5,icd9_6,icd9_7,icd9_8,icd9_9,icd9_10,icd9_11,icd9_12,icd9_13,icd9_14,icd9_15,icd9_16,icd9_17,icd9_18,icd9_19,icd9_20,age,Government,Medicaid,Medicare,Private,Self Pay,ELECTIVE,EMERGENCY,URGENT,F,M,CLINIC REFERRAL/PREMATURE,EMERGENCY ROOM ADMIT,Other_Adm_Location,PHYS REFERRAL/NORMAL DELI,HOME,HOME HEALTH CARE,Other_Discharge_Location,SNF,ENGL,Other_Language,CATHOLIC,Other_Religion,PROTESTANT QUAKER,Religion_missing,MARRIED,Other_Marital_Status,SINGLE,WIDOWED
0,165660,2134-05-10 11:30:00,2134-05-20 13:16:00,NaN,WHITE,2061-08-17 00:00:00,NaN,0,0,0,0,1,0,1,1,1,1,0,0,0,0,0,0,0,1,1,0,73,0,0,1,0,0,1,0,0,0,1,0,0,0,1,0,0,1,0,1,0,0,0,0,1,1,0,0,0
1,126055,2141-10-13 23:10:00,2141-11-03 18:45:00,NaN,BLACK/AFRICAN AMERICAN,2117-08-07 00:00:00,2142-08-30 00:00:00,0,0,1,1,0,1,1,1,1,1,0,0,1,0,1,1,0,0,1,0,24,0,1,0,0,0,0,1,0,1,0,0,1,0,0,1,0,0,0,1,0,0,0,0,1,0,0,1,0
2,136572,2142-06-18 03:11:00,2142-06-20 16:34:00,NaN,BLACK/AFRICAN AMERICAN,2117-08-07 00:00:00,2142-08-30 00:00:00,0,0,1,1,1,1,1,0,1,1,0,0,1,0,1,0,0,0,1,0,25,0,1,0,0,0,0,1,0,1,0,0,1,0,0,0,1,0,0,1,0,0,0,0,1,0,0,1,0
3,137510,2142-04-20 13:39:00,2142-04-22 15:30:00,NaN,BLACK/AFRICAN AMERICAN,2117-08-07 00:00:00,2142-08-30 00:00:00,0,0,1,1,0,1,1,0,0,1,0,0,1,0,1,0,0,0,1,0,25,0,1,0,0,0,0,1,0,1,0,0,1,0,0,0,1,0,0,1,0,0,0,0,1,0,0,1,0
4,158943,2142-03-21 09:25:00,2142-03-23 18:11:00,NaN,BLACK/AFRICAN AMERICAN,2117-08-07 00:00:00,2142-08-30 00:00:00,1,0,1,1,0,1,1,0,0,1,0,0,1,0,1,1,0,1,1,1,25,0,1,0,0,0,0,1,0,1,0,0,1,0,0,1,0,0,0,1,0,0,0,0,1,0,0,1,0


* CV

In [101]:
marital_value_counts_CV = final_formatted_static_CV["marital_status"].value_counts(dropna=False).rename_axis("unique_values").reset_index(name="counts")
marital_value_counts_CV

,unique_values,counts
0,MARRIED,12779
1,SINGLE,6002
2,WIDOWED,3802
3,DIVORCED,1575
4,NaN,1397
5,SEPARATED,292
6,UNKNOWN (DEFAULT),217
7,LIFE PARTNER,2


In [102]:
combined_marital_cv = list(marital_value_counts_CV[marital_value_counts_CV["counts"] < 2606]["unique_values"])
final_formatted_static_CV.loc[final_formatted_static_CV["marital_status"].isin(combined_marital_cv), "marital_status"] = "Other_Marital_Status"
final_formatted_static_CV["marital_status"].value_counts(dropna=False)

MARRIED                 12779
SINGLE                   6002
WIDOWED                  3802
Other_Marital_Status     3483
Name: marital_status, dtype: int64

In [103]:
marital_dummy_cv = pd.get_dummies(final_formatted_static_CV.marital_status)
final_formatted_static_CV = pd.concat([final_formatted_static_CV, marital_dummy_cv], axis=1)
final_formatted_static_CV.drop(columns="marital_status", inplace=True, axis=1)
with pd.option_context('display.max_rows', None, 'display.max_columns', None): 
    display(final_formatted_static_CV.head())

,hadm_id,admittime,dischtime,deathtime,ethnicity,dob,dod,icd9_1,icd9_2,icd9_3,icd9_4,icd9_5,icd9_6,icd9_7,icd9_8,icd9_9,icd9_10,icd9_11,icd9_12,icd9_13,icd9_14,icd9_15,icd9_16,icd9_17,icd9_18,icd9_19,icd9_20,age,Government,Medicaid,Medicare,Private,Self Pay,ELECTIVE,EMERGENCY,URGENT,F,M,EMERGENCY ROOM ADMIT,Other_Adm_Location,PHYS REFERRAL/NORMAL DELI,TRANSFER FROM HOSP/EXTRAM,HOME,HOME HEALTH CARE,Other_Discharge_Location,REHAB/DISTINCT PART HOSP,SNF,ENGL,Lan_Missing,Other_Language,CATHOLIC,JEWISH,Other_Religion,PROTESTANT QUAKER,Religion_missing,MARRIED,Other_Marital_Status,SINGLE,WIDOWED
0,141647,2183-08-20 17:25:00,2183-08-24 15:28:00,NaN,WHITE,2135-02-01 00:00:00,2184-02-22 00:00:00,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,48,0,0,0,1,0,0,1,0,1,0,0,1,0,0,1,0,0,0,0,0,1,0,1,0,0,0,0,0,0,1,0
1,172056,2143-03-03 09:25:00,2143-03-18 13:01:00,NaN,BLACK/AFRICAN AMERICAN,2116-06-27 00:00:00,NaN,1,0,0,0,1,0,1,1,0,0,0,0,1,0,1,0,0,1,0,0,27,0,1,0,0,0,0,1,0,1,0,1,0,0,0,0,0,0,1,0,1,0,0,0,0,1,0,0,0,0,1,0
2,142768,2164-11-18 07:00:00,2164-11-20 14:30:00,NaN,UNKNOWN/NOT SPECIFIED,2119-05-12 00:00:00,NaN,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,45,0,0,0,1,0,0,1,0,0,1,0,0,0,1,1,0,0,0,0,0,1,0,1,0,0,0,0,1,0,0,0
3,158569,2142-04-01 12:34:00,2142-04-08 14:46:00,NaN,UNKNOWN/NOT SPECIFIED,2076-09-15 00:00:00,2142-11-20 00:00:00,0,0,1,0,0,0,1,0,1,0,0,1,0,0,0,0,0,0,0,0,66,0,0,1,0,0,0,0,1,1,0,0,0,0,1,0,1,0,0,0,0,1,0,0,0,0,0,1,1,0,0,0
4,170324,2156-03-31 16:11:00,2156-04-29 15:05:00,NaN,WHITE,2117-10-02 00:00:00,NaN,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,1,1,0,39,1,0,0,0,0,0,1,0,0,1,1,0,0,0,1,0,0,0,0,1,0,0,1,0,0,0,0,1,0,0,0


## m) one-hot encoding categorical variable "ethnicity"

* MV

In [105]:
ethnicity_value_counts_MV = final_formatted_static_MV["ethnicity"].value_counts(dropna=False).rename_axis("unique_values").reset_index(name="counts")
combined_ethnicity_values_MV = list(ethnicity_value_counts_MV[ethnicity_value_counts_MV["counts"] < 2018]["unique_values"])
final_formatted_static_MV.loc[final_formatted_static_MV["ethnicity"].isin(combined_ethnicity_values_MV), "ethnicity"] = "Other_Ethnicity"
final_formatted_static_MV["ethnicity"].value_counts(dropna=False)

WHITE                     15110
Other_Ethnicity            3890
BLACK/AFRICAN AMERICAN     2018
Name: ethnicity, dtype: int64

In [106]:
ethnicity_dummies_mv = pd.get_dummies(final_formatted_static_MV.ethnicity)
final_formatted_static_MV = pd.concat([final_formatted_static_MV, ethnicity_dummies_mv], axis=1)
final_formatted_static_MV.drop(columns="ethnicity", axis=1, inplace=True)
with pd.option_context('display.max_rows', None, 'display.max_columns', None): 
    display(final_formatted_static_MV.head())

,hadm_id,admittime,dischtime,deathtime,dob,dod,icd9_1,icd9_2,icd9_3,icd9_4,icd9_5,icd9_6,icd9_7,icd9_8,icd9_9,icd9_10,icd9_11,icd9_12,icd9_13,icd9_14,icd9_15,icd9_16,icd9_17,icd9_18,icd9_19,icd9_20,age,Government,Medicaid,Medicare,Private,Self Pay,ELECTIVE,EMERGENCY,URGENT,F,M,CLINIC REFERRAL/PREMATURE,EMERGENCY ROOM ADMIT,Other_Adm_Location,PHYS REFERRAL/NORMAL DELI,HOME,HOME HEALTH CARE,Other_Discharge_Location,SNF,ENGL,Other_Language,CATHOLIC,Other_Religion,PROTESTANT QUAKER,Religion_missing,MARRIED,Other_Marital_Status,SINGLE,WIDOWED,BLACK/AFRICAN AMERICAN,Other_Ethnicity,WHITE
0,165660,2134-05-10 11:30:00,2134-05-20 13:16:00,NaN,2061-08-17 00:00:00,NaN,0,0,0,0,1,0,1,1,1,1,0,0,0,0,0,0,0,1,1,0,73,0,0,1,0,0,1,0,0,0,1,0,0,0,1,0,0,1,0,1,0,0,0,0,1,1,0,0,0,0,0,1
1,126055,2141-10-13 23:10:00,2141-11-03 18:45:00,NaN,2117-08-07 00:00:00,2142-08-30 00:00:00,0,0,1,1,0,1,1,1,1,1,0,0,1,0,1,1,0,0,1,0,24,0,1,0,0,0,0,1,0,1,0,0,1,0,0,1,0,0,0,1,0,0,0,0,1,0,0,1,0,1,0,0
2,136572,2142-06-18 03:11:00,2142-06-20 16:34:00,NaN,2117-08-07 00:00:00,2142-08-30 00:00:00,0,0,1,1,1,1,1,0,1,1,0,0,1,0,1,0,0,0,1,0,25,0,1,0,0,0,0,1,0,1,0,0,1,0,0,0,1,0,0,1,0,0,0,0,1,0,0,1,0,1,0,0
3,137510,2142-04-20 13:39:00,2142-04-22 15:30:00,NaN,2117-08-07 00:00:00,2142-08-30 00:00:00,0,0,1,1,0,1,1,0,0,1,0,0,1,0,1,0,0,0,1,0,25,0,1,0,0,0,0,1,0,1,0,0,1,0,0,0,1,0,0,1,0,0,0,0,1,0,0,1,0,1,0,0
4,158943,2142-03-21 09:25:00,2142-03-23 18:11:00,NaN,2117-08-07 00:00:00,2142-08-30 00:00:00,1,0,1,1,0,1,1,0,0,1,0,0,1,0,1,1,0,1,1,1,25,0,1,0,0,0,0,1,0,1,0,0,1,0,0,1,0,0,0,1,0,0,0,0,1,0,0,1,0,1,0,0


* CV

In [108]:
ethnicity_value_counts_CV = final_formatted_static_CV["ethnicity"].value_counts(dropna=False).rename_axis("unique_values").reset_index(name="counts")
# ethnicity_value_counts_CV
combined_ethnicity_values_CV = ethnicity_value_counts_CV[ethnicity_value_counts_CV["counts"] < 2606]["unique_values"]
# transfer value "UNKNOWN/NOT SPECIFIED" into "Ethnicity_Missing"
final_formatted_static_CV.loc[final_formatted_static_CV["ethnicity"]=="UNKNOWN/NOT SPECIFIED", "ethnicity"] = "Ethnicity_Missing"
final_formatted_static_CV.loc[final_formatted_static_CV["ethnicity"].isin(combined_ethnicity_values_CV), "ethnicity"] = "Other_Ethnicity"
final_formatted_static_CV["ethnicity"].value_counts(dropna=False)

WHITE                18301
Other_Ethnicity       4590
Ethnicity_Missing     3175
Name: ethnicity, dtype: int64

In [109]:
ethnicity_dummies_cv = pd.get_dummies(final_formatted_static_CV.ethnicity)
final_formatted_static_CV = pd.concat([final_formatted_static_CV, ethnicity_dummies_cv], axis=1)
final_formatted_static_CV.drop(columns="ethnicity", axis=1, inplace=True)
with pd.option_context('display.max_rows', None, 'display.max_columns', None): 
    display(final_formatted_static_CV.head())

,hadm_id,admittime,dischtime,deathtime,dob,dod,icd9_1,icd9_2,icd9_3,icd9_4,icd9_5,icd9_6,icd9_7,icd9_8,icd9_9,icd9_10,icd9_11,icd9_12,icd9_13,icd9_14,icd9_15,icd9_16,icd9_17,icd9_18,icd9_19,icd9_20,age,Government,Medicaid,Medicare,Private,Self Pay,ELECTIVE,EMERGENCY,URGENT,F,M,EMERGENCY ROOM ADMIT,Other_Adm_Location,PHYS REFERRAL/NORMAL DELI,TRANSFER FROM HOSP/EXTRAM,HOME,HOME HEALTH CARE,Other_Discharge_Location,REHAB/DISTINCT PART HOSP,SNF,ENGL,Lan_Missing,Other_Language,CATHOLIC,JEWISH,Other_Religion,PROTESTANT QUAKER,Religion_missing,MARRIED,Other_Marital_Status,SINGLE,WIDOWED,Ethnicity_Missing,Other_Ethnicity,WHITE
0,141647,2183-08-20 17:25:00,2183-08-24 15:28:00,NaN,2135-02-01 00:00:00,2184-02-22 00:00:00,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,48,0,0,0,1,0,0,1,0,1,0,0,1,0,0,1,0,0,0,0,0,1,0,1,0,0,0,0,0,0,1,0,0,0,1
1,172056,2143-03-03 09:25:00,2143-03-18 13:01:00,NaN,2116-06-27 00:00:00,NaN,1,0,0,0,1,0,1,1,0,0,0,0,1,0,1,0,0,1,0,0,27,0,1,0,0,0,0,1,0,1,0,1,0,0,0,0,0,0,1,0,1,0,0,0,0,1,0,0,0,0,1,0,0,1,0
2,142768,2164-11-18 07:00:00,2164-11-20 14:30:00,NaN,2119-05-12 00:00:00,NaN,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,45,0,0,0,1,0,0,1,0,0,1,0,0,0,1,1,0,0,0,0,0,1,0,1,0,0,0,0,1,0,0,0,1,0,0
3,158569,2142-04-01 12:34:00,2142-04-08 14:46:00,NaN,2076-09-15 00:00:00,2142-11-20 00:00:00,0,0,1,0,0,0,1,0,1,0,0,1,0,0,0,0,0,0,0,0,66,0,0,1,0,0,0,0,1,1,0,0,0,0,1,0,1,0,0,0,0,1,0,0,0,0,0,1,1,0,0,0,1,0,0
4,170324,2156-03-31 16:11:00,2156-04-29 15:05:00,NaN,2117-10-02 00:00:00,NaN,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,1,1,0,39,1,0,0,0,0,0,1,0,0,1,1,0,0,0,1,0,0,0,0,1,0,0,1,0,0,0,0,1,0,0,0,0,0,1


# Finally drop the columns: ["admittime", "dischtime", "deathtime", "dob", "dod"], and storage the static data into files: "../Processed_Data/mimic_static_CV" & "../Processed_Data/mimic_static_MV"

In [110]:
final_formatted_static_MV.drop(columns=["admittime", "dischtime", "deathtime", "dob", "dod"], inplace=True, axis=1)
final_formatted_static_CV.drop(columns=["admittime", "dischtime", "deathtime", "dob", "dod"], inplace=True, axis=1)

In [111]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None): 
    display(final_formatted_static_CV.head())

,hadm_id,icd9_1,icd9_2,icd9_3,icd9_4,icd9_5,icd9_6,icd9_7,icd9_8,icd9_9,icd9_10,icd9_11,icd9_12,icd9_13,icd9_14,icd9_15,icd9_16,icd9_17,icd9_18,icd9_19,icd9_20,age,Government,Medicaid,Medicare,Private,Self Pay,ELECTIVE,EMERGENCY,URGENT,F,M,EMERGENCY ROOM ADMIT,Other_Adm_Location,PHYS REFERRAL/NORMAL DELI,TRANSFER FROM HOSP/EXTRAM,HOME,HOME HEALTH CARE,Other_Discharge_Location,REHAB/DISTINCT PART HOSP,SNF,ENGL,Lan_Missing,Other_Language,CATHOLIC,JEWISH,Other_Religion,PROTESTANT QUAKER,Religion_missing,MARRIED,Other_Marital_Status,SINGLE,WIDOWED,Ethnicity_Missing,Other_Ethnicity,WHITE
0,141647,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,48,0,0,0,1,0,0,1,0,1,0,0,1,0,0,1,0,0,0,0,0,1,0,1,0,0,0,0,0,0,1,0,0,0,1
1,172056,1,0,0,0,1,0,1,1,0,0,0,0,1,0,1,0,0,1,0,0,27,0,1,0,0,0,0,1,0,1,0,1,0,0,0,0,0,0,1,0,1,0,0,0,0,1,0,0,0,0,1,0,0,1,0
2,142768,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,45,0,0,0,1,0,0,1,0,0,1,0,0,0,1,1,0,0,0,0,0,1,0,1,0,0,0,0,1,0,0,0,1,0,0
3,158569,0,0,1,0,0,0,1,0,1,0,0,1,0,0,0,0,0,0,0,0,66,0,0,1,0,0,0,0,1,1,0,0,0,0,1,0,1,0,0,0,0,1,0,0,0,0,0,1,1,0,0,0,1,0,0
4,170324,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,1,1,0,39,1,0,0,0,0,0,1,0,0,1,1,0,0,0,1,0,0,0,0,1,0,0,1,0,0,0,0,1,0,0,0,0,0,1


In [112]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None): 
    display(final_formatted_static_MV.head())

,hadm_id,icd9_1,icd9_2,icd9_3,icd9_4,icd9_5,icd9_6,icd9_7,icd9_8,icd9_9,icd9_10,icd9_11,icd9_12,icd9_13,icd9_14,icd9_15,icd9_16,icd9_17,icd9_18,icd9_19,icd9_20,age,Government,Medicaid,Medicare,Private,Self Pay,ELECTIVE,EMERGENCY,URGENT,F,M,CLINIC REFERRAL/PREMATURE,EMERGENCY ROOM ADMIT,Other_Adm_Location,PHYS REFERRAL/NORMAL DELI,HOME,HOME HEALTH CARE,Other_Discharge_Location,SNF,ENGL,Other_Language,CATHOLIC,Other_Religion,PROTESTANT QUAKER,Religion_missing,MARRIED,Other_Marital_Status,SINGLE,WIDOWED,BLACK/AFRICAN AMERICAN,Other_Ethnicity,WHITE
0,165660,0,0,0,0,1,0,1,1,1,1,0,0,0,0,0,0,0,1,1,0,73,0,0,1,0,0,1,0,0,0,1,0,0,0,1,0,0,1,0,1,0,0,0,0,1,1,0,0,0,0,0,1
1,126055,0,0,1,1,0,1,1,1,1,1,0,0,1,0,1,1,0,0,1,0,24,0,1,0,0,0,0,1,0,1,0,0,1,0,0,1,0,0,0,1,0,0,0,0,1,0,0,1,0,1,0,0
2,136572,0,0,1,1,1,1,1,0,1,1,0,0,1,0,1,0,0,0,1,0,25,0,1,0,0,0,0,1,0,1,0,0,1,0,0,0,1,0,0,1,0,0,0,0,1,0,0,1,0,1,0,0
3,137510,0,0,1,1,0,1,1,0,0,1,0,0,1,0,1,0,0,0,1,0,25,0,1,0,0,0,0,1,0,1,0,0,1,0,0,0,1,0,0,1,0,0,0,0,1,0,0,1,0,1,0,0
4,158943,1,0,1,1,0,1,1,0,0,1,0,0,1,0,1,1,0,1,1,1,25,0,1,0,0,0,0,1,0,1,0,0,1,0,0,1,0,0,0,1,0,0,0,0,1,0,0,1,0,1,0,0


In [113]:
final_formatted_static_CV.shape

(26066, 56)

In [114]:
final_formatted_static_MV.shape

(21018, 53)

In [115]:
final_formatted_static_CV.to_csv("../Processed_Data/mimic_static_CV.csv", index=False)

In [116]:
final_formatted_static_MV.to_csv("../Processed_Data/mimic_static_MV.csv", index=False)